## 공통 모듈 불러와서 사용하기

In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
from common.hello import *
from common.yfinance import *

hello()

Hello MS AI School Project Team 6


## 필요한 라이브러리 import, 그래프 출력 한글 처리

In [2]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import gradio as gr
from io import BytesIO
from PIL import Image

import platform
from matplotlib import font_manager, rc

import random
import base64

def set_korean_font():
    if platform.system() == 'Windows':
        # Windows의 경우
        font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
        rc('font', family=font_name)
    elif platform.system() == 'Darwin':
        # macOS의 경우
        rc('font', family='AppleGothic')
    else:
        # 기타 OS의 경우
        rc('font', family='NanumGothic')

# 한글 폰트 설정
set_korean_font()

/opt/anaconda3/envs/py_3_12_3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 주식 데이터를 yfinance 에서 가져와서 데이터프레임과 그래프로 출력하는 함수

In [ ]:
''' 공통 모듈로 분리
# 주식 데이터를 가져오고 차트를 생성하는 함수
def get_stock_data(ticker):
    # yfinance를 이용해 주식 데이터 가져오기 (최근 1년 데이터)
    data = yf.download(ticker, period="1y")
    
    # 결과가 없으면 안내 문구 반환
    if data.empty:
        return "데이터가 없습니다. 유효한 티커를 입력하세요.", None
    
    # 데이터프레임을 정리하여 표시할 정보 선택
    data_display = data[['Open', 'High', 'Low', 'Close', 'Volume']]
    
    # 열 이름을 문자열로 설정하여 문제 해결
    data_display.columns = [col[0] if isinstance(col, tuple) else col for col in data_display.columns]
    
    # 인덱스를 열로 변환
    data_display.reset_index(inplace=True)
    
    # 날짜 정보를 년월일 정보만 출력
    data_display['Date'] = data_display['Date'].dt.strftime('%Y-%m-%d')
    
    # 나머지 값들은 소수점 둘째자리까지 출력하고 천단위 컴마(,)를 붙여줌
    # data_display = data_display.round(2)
    data_display['Open'] = data_display['Open'].apply(lambda x: f"{x:,.2f}")
    data_display['High'] = data_display['High'].apply(lambda x: f"{x:,.2f}")
    data_display['Low'] = data_display['Low'].apply(lambda x: f"{x:,.2f}")
    data_display['Close'] = data_display['Close'].apply(lambda x: f"{x:,.2f}")
    data_display['Volume'] = data_display['Volume'].apply(lambda x: f"{x:,}")
    
    
    # 차트 생성
    plt.figure(figsize=(10, 5))
    plt.plot(data['Close'], label="Close Price", color='blue')
    plt.title(f"{ticker} - 일별 종가 차트")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.grid()
    
    # 이미지를 메모리에 저장
    buf = BytesIO()
    plt.savefig(buf, format="png")
    plt.close()
    buf.seek(0)  # 버퍼의 시작으로 이동
    img = Image.open(buf)  # PIL 이미지로 변환
    
    data_display = data_display.iloc[::-1].reset_index(drop=True)
    return data_display, img
'''

## Gradio UI 구성

In [ ]:
# Gradio 인터페이스 설정
with gr.Blocks(gr.themes.Monochrome()) as demo:
    gr.Markdown("# 주식 데이터 조회")
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder="예: AAPL, MSFT 등")
    with gr.Row():
        submit_btn = gr.Button("조회")
        reset_btn = gr.Button("초기화")
    data_output = gr.Dataframe(label="일별 주가 데이터", visible=False) 
    chart_output = gr.Image(label="주가 차트", visible=False)
    
    def update_visibility(ticker):
        ticker = ticker.upper()  # Convert ticker to uppercase
        data, img = get_yfinance_stock_data(ticker)
        return gr.update(visible=True), gr.update(visible=True), data, img
    
    def reset_fields():
        return "", gr.update(visible=False), gr.update(visible=False)
    
    submit_btn.click(update_visibility, inputs=ticker_input, outputs=[data_output, chart_output, data_output, chart_output])
    ticker_input.submit(update_visibility, inputs=ticker_input, outputs=[data_output, chart_output, data_output, chart_output])
    reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, data_output, chart_output])

# Gradio 앱 실행
demo.launch()
# demo.launch(share=True)

## 조회 기간 설정하여 조회되도록 수정

In [ ]:
''' 공통 모듈로 분리
# 주식 데이터를 가져오고 차트를 생성하는 함수
def get_stock_data(ticker, start, end):
    # yfinance를 이용해 주식 데이터 가져오기 (지정된 기간 데이터)
    data = yf.download(ticker, start=start, end=end)
    
    # 결과가 없으면 안내 문구 반환
    if data.empty:
        return "데이터가 없습니다. 유효한 티커를 입력하세요.", None
    
    # 데이터프레임을 정리하여 표시할 정보 선택
    data_display = data[['Open', 'High', 'Low', 'Close', 'Volume']]
    
    # 열 이름을 문자열로 설정하여 문제 해결
    data_display.columns = [col[0] if isinstance(col, tuple) else col for col in data_display.columns]
    
    # 인덱스를 열로 변환
    data_display.reset_index(inplace=True)
    
    # 날짜 정보를 년월일 정보만 출력
    data_display['Date'] = data_display['Date'].dt.strftime('%Y-%m-%d')
    
    # 나머지 값들은 소수점 둘째자리까지 출력하고 천단위 컴마(,)를 붙여줌
    data_display['Open'] = data_display['Open'].apply(lambda x: f"{x:,.2f}")
    data_display['High'] = data_display['High'].apply(lambda x: f"{x:,.2f}")
    data_display['Low'] = data_display['Low'].apply(lambda x: f"{x:,.2f}")
    data_display['Close'] = data_display['Close'].apply(lambda x: f"{x:,.2f}")
    data_display['Volume'] = data_display['Volume'].apply(lambda x: f"{x:,}")
    
    # 차트 생성
    plt.figure(figsize=(10, 5))
    plt.plot(data['Close'], label="Close Price", color='blue')
    plt.title(f"{ticker} - 일별 종가 차트")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.grid()
    
    # 이미지를 메모리에 저장
    buf = BytesIO()
    plt.savefig(buf, format="png")
    plt.close()
    buf.seek(0)  # 버퍼의 시작으로 이동
    img = Image.open(buf)  # PIL 이미지로 변환
    
    data_display = data_display.iloc[::-1].reset_index(drop=True)
    return data_display, img
'''

## 시작일 종료일 지정하는 UI

In [ ]:
# Gradio 인터페이스 설정
with gr.Blocks(gr.themes.Monochrome()) as demo:
    gr.Markdown("# 주식 데이터 조회")
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder="예: AAPL, MSFT 등")
    start_date = gr.Textbox(label="조회 시작일", placeholder="예: 2022-01-01")
    end_date = gr.Textbox(label="조회 종료일", placeholder="예: 2023-01-01")
    with gr.Row():
        submit_btn = gr.Button("조회")
        reset_btn = gr.Button("초기화")
    data_output = gr.Dataframe(label="일별 주가 데이터", visible=False) 
    chart_output = gr.Image(label="주가 차트", visible=False)
    
    def update_visibility(ticker, start_date, end_date):
        ticker = ticker.upper()  # Convert ticker to uppercase
        data, img = get_yfinance_stock_data(ticker, start=start_date, end=end_date)
        return gr.update(visible=True), gr.update(visible=True), data, img
    
    def reset_fields():
        return "", "", "", gr.update(visible=False), gr.update(visible=False)
    
    submit_btn.click(update_visibility, inputs=[ticker_input, start_date, end_date], outputs=[data_output, chart_output, data_output, chart_output])
    ticker_input.submit(update_visibility, inputs=[ticker_input, start_date, end_date], outputs=[data_output, chart_output, data_output, chart_output])
    reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, start_date, end_date, data_output, chart_output])

# Gradio 앱 실행
demo.launch()
# demo.launch(share=True)

In [ ]:
start_date = "2022-01-01"
end_date = "2022-01-10"
get_yfinance_stock_data("AAPL", start=start_date, end=end_date)

## Gradio UI 연습, 2개의 종목을 한번에 조회

In [ ]:
'''
# Gradio 인터페이스 설정
with gr.Blocks(gr.themes.Monochrome()) as demo:
    gr.Markdown("# 주식 데이터 조회")
    with gr.Row():
        ticker_input1 = gr.Textbox(label="첫 번째 주식 티커 입력", placeholder="예: AAPL")
        ticker_input2 = gr.Textbox(label="두 번째 주식 티커 입력", placeholder="예: MSFT")
    with gr.Row():
        submit_btn = gr.Button("조회")
        reset_btn = gr.Button("초기화")
    with gr.Row():
        data_output1 = gr.Dataframe(label="첫 번째 종목 일별 주가 데이터", visible=False)
        data_output2 = gr.Dataframe(label="두 번째 종목 일별 주가 데이터", visible=False)
    with gr.Row():
        chart_output1 = gr.Image(label="첫 번째 종목 주가 차트", visible=False)
        chart_output2 = gr.Image(label="두 번째 종목 주가 차트", visible=False)
    
    def update_visibility(ticker1, ticker2):
        data1, img1 = get_stock_data(ticker1) if ticker1 else (None, None)
        data2, img2 = get_stock_data(ticker2) if ticker2 else (None, None)
        
        return (
            gr.update(visible=True) if data1 is not None else gr.update(visible=False),
            gr.update(visible=True) if data2 is not None else gr.update(visible=False),
            gr.update(visible=True) if img1 is not None else gr.update(visible=False),
            gr.update(visible=True) if img2 is not None else gr.update(visible=False),
            data1 if data1 is not None else gr.update(visible=False),
            data2 if data2 is not None else gr.update(visible=False),
            img1,
            img2
        )
    
    def reset_fields():
        return "", "", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)
    
    submit_btn.click(update_visibility, inputs=[ticker_input1, ticker_input2], outputs=[data_output1, data_output2, chart_output1, chart_output2, data_output1, data_output2, chart_output1, chart_output2])
    ticker_input1.submit(update_visibility, inputs=[ticker_input1, ticker_input2], outputs=[data_output1, data_output2, chart_output1, chart_output2, data_output1, data_output2, chart_output1, chart_output2])
    ticker_input2.submit(update_visibility, inputs=[ticker_input1, ticker_input2], outputs=[data_output1, data_output2, chart_output1, chart_output2, data_output1, data_output2, chart_output1, chart_output2])
    reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input1, ticker_input2, data_output1, data_output2, chart_output1, chart_output2])

# Gradio 앱 실행
demo.launch()
# demo.launch(share=True)
'''

## 이전 조회한 종목을 바로 선택할 수 있는 UI

In [ ]:
# Gradio 인터페이스 설정
with gr.Blocks(gr.themes.Monochrome()) as demo:
    previous_tickers = gr.State(set())
    gr.Markdown("# 주식 데이터 조회")
    previous_tickers_display = gr.Radio(label="이전 조회된 티커", choices=[], interactive=True, visible=False)
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder="예: AAPL, MSFT 등")
    warning_message = gr.Markdown(value="", visible=False)
    with gr.Row():
        submit_btn = gr.Button("조회")
        reset_btn = gr.Button("초기화")
    data_output = gr.Dataframe(label="일별 주가 데이터", visible=False, render=False) 
    chart_output = gr.Image(label="주가 차트", visible=False)
    
    def update_visibility(ticker, previous_tickers):
        if not ticker:
            return gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="티커를 입력하세요.", visible=True)
        
        ticker = ticker.upper()  # Convert ticker to uppercase
        data, img = get_yfinance_stock_data(ticker)
        previous_tickers.add(ticker)
        return (
            gr.update(visible=True, label=f"일별 주가 데이터 [{ticker}]"),
            gr.update(visible=True),
            data,
            img,
            gr.update(choices=list(previous_tickers), visible=True),
            gr.update(visible=False)
        )
    
    def reset_fields():
        return "", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)
    
    def update_ticker_input(selected_ticker):
        return selected_ticker, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(value="", visible=False)
    
    def auto_submit_ticker(selected_ticker, previous_tickers):
        return update_visibility(selected_ticker, previous_tickers)
    
    submit_btn.click(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, data_output, chart_output, previous_tickers_display, warning_message])
    ticker_input.submit(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, data_output, chart_output, previous_tickers_display, warning_message])
    reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, data_output, chart_output, warning_message])
    previous_tickers_display.change(update_ticker_input, inputs=previous_tickers_display, outputs=[ticker_input, data_output, chart_output, warning_message])
    previous_tickers_display.change(auto_submit_ticker, inputs=[previous_tickers_display, previous_tickers], outputs=[data_output, chart_output, data_output, chart_output, previous_tickers_display, warning_message])

# Gradio 앱 실행
demo.launch()
# demo.launch(share=True)


## 가격에측 가능한(모델 학습이 완료된) 티커에 대해서만 조회 되도록 수정

In [ ]:
import random

# 사용 가능한 티커 목록 가져오기
available_tickers = pd.read_csv('../data/available_tickers.csv')['tickers'].str.upper().tolist()
random.shuffle(available_tickers)

# Gradio 인터페이스 설정
with gr.Blocks(gr.themes.Monochrome()) as demo:
    previous_tickers = gr.State(set())
    gr.Markdown("# 주식 데이터 조회")
    previous_tickers_display = gr.Radio(label="이전 조회된 티커", choices=[], interactive=True, visible=False)
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder=f"예: {", ".join(available_tickers[:5])} 등")
    warning_message = gr.Markdown(value="", visible=False)
    with gr.Row():
        submit_btn = gr.Button("조회")
        reset_btn = gr.Button("초기화")

    chart_output = gr.Image(label="주가 차트", visible=False)
    data_output = gr.Dataframe(label="일별 주가 데이터", visible=False)
    
    def update_visibility(ticker, previous_tickers):
        if not ticker:
            return gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="티커를 입력하세요.", visible=True)
        
        ticker = ticker.upper()  # Convert ticker to uppercase
        if ticker.upper() not in available_tickers:
            return gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="입력하신 종목의 가격 예측은 추후에 제공될 예정입니다..", visible=True)
        
        data, img = get_yfinance_stock_data(ticker)
        previous_tickers.add(ticker)
        return (
            gr.update(visible=True, label=f"일별 주가 데이터 [{ticker}]"),
            gr.update(visible=True),
            data,
            img,
            gr.update(choices=list(previous_tickers), visible=True),
            gr.update(visible=False)
        )
    
    def reset_fields():
        return "", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)
    
    def update_ticker_input(selected_ticker):
        return selected_ticker, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(value="", visible=False)
    
    def auto_submit_ticker(selected_ticker, previous_tickers):
        return update_visibility(selected_ticker, previous_tickers)
    
    submit_btn.click(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, data_output, chart_output, previous_tickers_display, warning_message])
    ticker_input.submit(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, data_output, chart_output, previous_tickers_display, warning_message])
    reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, data_output, chart_output, warning_message])
    previous_tickers_display.change(update_ticker_input, inputs=previous_tickers_display, outputs=[ticker_input, data_output, chart_output, warning_message])
    previous_tickers_display.change(auto_submit_ticker, inputs=[previous_tickers_display, previous_tickers], outputs=[data_output, chart_output, data_output, chart_output, previous_tickers_display, warning_message])

# Gradio 앱 실행
demo.launch()
# demo.launch(share=True)


## 배경 이미지 추가

In [ ]:
import random
import base64

# 사용 가능한 티커 목록 가져오기
available_tickers = pd.read_csv('../data/available_tickers.csv')['tickers'].str.upper().tolist()
random.shuffle(available_tickers)

# 이미지 파일을 base64로 인코딩
with open("images/bg_01.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode()

# Gradio 인터페이스 설정
css_code = f"""
.gradio-container {{
    background-image: url(data:image/png;base64,{encoded_string});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 0.8;
}}

.custom-markdown h1, p {{
    color: white;
}}

.table-wrap {{
    max-height: 400px !important;
    overflow-y: auto !important;
}}
"""

with gr.Blocks(gr.themes.Monochrome(), css=css_code) as demo:
# with gr.Blocks(gr.themes.Default(primary_hue=gr.themes.colors.red, secondary_hue=gr.themes.colors.pink)) as demo:
    previous_tickers = gr.State(set())
    gr.Markdown("# 주식 가격 예측", elem_classes="custom-markdown")
    previous_tickers_display = gr.Radio(label="이전 조회된 티커", choices=[], interactive=True, visible=False)
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder=f"예: {", ".join(available_tickers[:5])} 등")
    warning_message = gr.Markdown(value="", visible=False)
    with gr.Row():
        submit_btn = gr.Button("조회")
        reset_btn = gr.Button("초기화")

    chart_output = gr.Image(label="주가 차트", visible=False)
    data_output = gr.Dataframe(label="일별 주가 데이터", visible=False)
    
    def update_visibility(ticker, previous_tickers):
        if not ticker:
            return gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="티커를 입력하세요.", visible=True, elem_classes="custom-markdown")
        
        ticker = ticker.upper()  # Convert ticker to uppercase
        if ticker.upper() not in available_tickers:
            return gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="입력하신 종목의 가격 예측은 추후에 제공될 예정입니다..", visible=True)
        
        data, img = get_yfinance_stock_data(ticker)
        previous_tickers.add(ticker)
        return (
            gr.update(visible=True, label=f"일별 주가 데이터 [{ticker}]"),
            gr.update(visible=True),
            data,
            img,
            gr.update(choices=list(previous_tickers), visible=True),
            gr.update(visible=False)
        )
    
    def reset_fields():
        return "", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)
    
    def update_ticker_input(selected_ticker):
        return selected_ticker, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(value="", visible=False)
    
    def auto_submit_ticker(selected_ticker, previous_tickers):
        return update_visibility(selected_ticker, previous_tickers)
    
    submit_btn.click(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, data_output, chart_output, previous_tickers_display, warning_message])
    ticker_input.submit(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, data_output, chart_output, previous_tickers_display, warning_message])
    reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, data_output, chart_output, warning_message])
    previous_tickers_display.change(update_ticker_input, inputs=previous_tickers_display, outputs=[ticker_input, data_output, chart_output, warning_message])
    previous_tickers_display.change(auto_submit_ticker, inputs=[previous_tickers_display, previous_tickers], outputs=[data_output, chart_output, data_output, chart_output, previous_tickers_display, warning_message])

# Gradio 앱 실행
# demo.launch()
demo.launch(share=True)


## 백엔드 서비스 연동

In [ ]:
# 사용 가능한 티커 목록 가져오기
available_tickers = pd.read_csv('../data/available_tickers.csv')['tickers'].str.upper().tolist()
random.shuffle(available_tickers)

# 이미지 파일을 base64로 인코딩
with open("images/bg_01.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode()

# Gradio 인터페이스 설정
css_code = f"""
.gradio-container {{
    background-image: url(data:image/png;base64,{encoded_string});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 0.8;
}}

.custom-markdown h1, p {{
    color: white;
}}

div.custom-title > div > p {{
    color: white;
}}

.table-wrap {{
    max-height: 400px !important;
    overflow-y: auto !important;
}}
"""

with gr.Blocks(gr.themes.Monochrome(), css=css_code) as demo:
# with gr.Blocks(gr.themes.Default(primary_hue=gr.themes.colors.red, secondary_hue=gr.themes.colors.pink)) as demo:
    previous_tickers = gr.State(set())
    gr.Markdown("# 주식 가격 예측", elem_classes="custom-markdown")
    previous_tickers_display = gr.Radio(label="이전 조회된 티커", choices=[], interactive=True, visible=False)
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder=f"예: {", ".join(available_tickers[:5])} 등")
    warning_message = gr.Markdown(value="", visible=False)
    with gr.Row():
        submit_btn = gr.Button("조회")
        reset_btn = gr.Button("초기화")

    chart_output = gr.Image(label="주가 차트", elem_classes="custom-markdown", visible=False)
    data_output = gr.Dataframe(label="일별 주가 데이터", elem_classes="custom-title", visible=False)
    
    def update_visibility(ticker, previous_tickers):
        if not ticker:
            return gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="티커를 입력하세요.", visible=True, elem_classes="custom-markdown")
        
        ticker = ticker.upper()  # Convert ticker to uppercase
        if ticker.upper() not in available_tickers:
            return gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="입력하신 종목의 가격 예측은 추후에 제공될 예정입니다..", visible=True)
        
        data, img = predict_stock_price2(ticker)
        previous_tickers.add(ticker)
        return (
            gr.update(visible=True, label=f"일별 주가 데이터 [{ticker}]", elem_classes="custom-title"),
            gr.update(visible=True),
            data,
            img,
            gr.update(choices=list(previous_tickers), visible=True),
            gr.update(visible=False)
        )
    
    def reset_fields():
        return "", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)
    
    def update_ticker_input(selected_ticker):
        return selected_ticker, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(value="", visible=False)
    
    def auto_submit_ticker(selected_ticker, previous_tickers):
        return update_visibility(selected_ticker, previous_tickers)
    
    submit_btn.click(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, data_output, chart_output, previous_tickers_display, warning_message])
    ticker_input.submit(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, data_output, chart_output, previous_tickers_display, warning_message])
    reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, data_output, chart_output, warning_message])
    previous_tickers_display.change(update_ticker_input, inputs=previous_tickers_display, outputs=[ticker_input, data_output, chart_output, warning_message])
    previous_tickers_display.change(auto_submit_ticker, inputs=[previous_tickers_display, previous_tickers], outputs=[data_output, chart_output, data_output, chart_output, previous_tickers_display, warning_message])

# Gradio 앱 실행
demo.launch()
# demo.launch(share=True)


In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
from common.hello import *
from common.yfinance import *

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import gradio as gr
from io import BytesIO
from PIL import Image

import platform
from matplotlib import font_manager, rc

import random
import base64

def set_korean_font():
    if platform.system() == 'Windows':
        # Windows의 경우
        font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
        rc('font', family=font_name)
    elif platform.system() == 'Darwin':
        # macOS의 경우
        rc('font', family='AppleGothic')
    else:
        # 기타 OS의 경우
        rc('font', family='NanumGothic')

# 한글 폰트 설정
set_korean_font()

## 주가 예측 정보 시각화 추가

In [ ]:
# 사용 가능한 티커 목록 가져오기
available_tickers = pd.read_csv('../data/available_tickers.csv')['tickers'].str.upper().tolist()
random.shuffle(available_tickers)

# 이미지 파일을 base64로 인코딩
with open("images/bg_01.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode()

with open("images/up.jpeg", "rb") as image_file:
    encoded_up_image = base64.b64encode(image_file.read()).decode()

with open("images/down.jpeg", "rb") as image_file:
    encoded_down_image = base64.b64encode(image_file.read()).decode()
    
# Gradio 인터페이스 설정
css_code = f"""
.gradio-container {{
    background-image: url(data:image/png;base64,{encoded_string});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 0.8;
}}

.custom-markdown h1, p {{
    color: white;
}}

div.custom-title > div > p {{
    color: white;
}}

.table-wrap {{
    max-height: 400px !important;
    overflow-y: auto !important;
}}

.predict_wrapper {{
    display: flex;
    justify-content: space-between;
}}

.predict_result {{
    width: 50%;
    height: 200px;
    background-color: rgba(255, 255, 255, 0.5);
    border-radius: 10px;
    margin-left: 10px;
    padding: 10px;
}}

.predict_result_up {{
    width: 50%;
    height: 200px;
    border-radius: 10px;
    background-image: url(data:image/png;base64,{encoded_up_image});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 1;
}}

.predict_result_down {{
    width: 50%;
    height: 200px;
    border-radius: 10px;
    background-image: url(data:image/png;base64,{encoded_down_image});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 1;
}}

.predict_ticker, .predict_price, .start_price {{
    padding: 10px;
    font-size: 20px;
}}
"""

with gr.Blocks(gr.themes.Monochrome(), css=css_code) as demo:
    previous_tickers = gr.State(set())
    gr.Markdown("# 주식 가격 예측", elem_classes="custom-markdown")
    previous_tickers_display = gr.Radio(label="이전 조회된 티커", choices=[], interactive=True, visible=False)
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder=f"예: {", ".join(available_tickers[:5])} 등")
    warning_message = gr.Markdown(value="", visible=False)
    with gr.Row():
        submit_btn = gr.Button("조회")
        reset_btn = gr.Button("초기화")

    predict_result_up = gr.HTML(
        """
        <div class='predict_wrapper'>
            <div class='predict_result_up'>

            </div>
            <div class='predict_result'>
                {ticker}
            </div>
        </div>
        """,
        visible=False
    )

    predict_result_down = gr.HTML(
        """
        <div class='predict_wrapper'>
            <div class='predict_result_down'>

            </div>
            <div class='predict_result'>
                {ticker}
            </div>
        </div>
        """,
        visible=False
    )

    chart_output = gr.Image(label="주가 차트", elem_classes="custom-markdown", visible=False)
    data_output = gr.Dataframe(label="일별 주가 데이터", elem_classes="custom-title", visible=False)

    def update_visibility(ticker, previous_tickers):
        if not ticker:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="티커를 입력하세요.", visible=True, elem_classes="custom-markdown")

        ticker = ticker.upper()  # Convert ticker to uppercase
        if ticker.upper() not in available_tickers:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="입력하신 종목의 가격 예측은 추후에 제공될 예정입니다..", visible=True)

        data, img, isUp = predict_stock_price2(ticker)
        
        # 임시코드: AAPL이면 True, 그 외에는 False
        if ticker == "AAPL":
            isUp = True
        else:
            isUp = False

        previous_tickers.add(ticker)
        return (
            gr.update(visible=True, label=f"일별 주가 데이터 [{ticker}]", elem_classes="custom-title"),
            gr.update(visible=True),
            gr.update(visible=isUp, value=f"""
            <div class='predict_wrapper'>
                <div class='predict_result_up'>

                </div>
                <div class='predict_result'>
                    <div class='predict_ticker'>
                        Ticker: {ticker}
                    </div>  
                    <div class='predict_price'>
                        Predicted Price: 234.56
                    </div>
                    <div class='start_price'>
                        Start Price : 231.45
                    </div>  
                </div>
            </div>
            """),
            gr.update(visible=not isUp, value=f"""
            <div class='predict_wrapper'>
                <div class='predict_result_down'>

                </div>
                <div class='predict_result'>
                    <div class='predict_ticker'>
                        Ticker: {ticker}
                    </div>  
                    <div class='predict_price'>
                        Predicted Price: 231.45
                    </div>
                    <div class='start_price'>
                        Start Price : 234.56
                    </div>  
                </div>
            </div>
            """),
            data,
            img,
            gr.update(choices=list(previous_tickers), visible=True),
            gr.update(visible=False)
        )

    def reset_fields():
        return "", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)

    def update_ticker_input(selected_ticker):
        return selected_ticker, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(value="", visible=False)

    def auto_submit_ticker(selected_ticker, previous_tickers):
        return update_visibility(selected_ticker, previous_tickers)

    submit_btn.click(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output, previous_tickers_display, warning_message])
    ticker_input.submit(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output, previous_tickers_display, warning_message])
    reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message])
    previous_tickers_display.change(update_ticker_input, inputs=previous_tickers_display, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message])
    previous_tickers_display.change(auto_submit_ticker, inputs=[previous_tickers_display, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output, previous_tickers_display, warning_message])

# Gradio 앱 실행
demo.launch()
# demo.launch(share=True)

In [ ]:
import gradio as gr
import plotly.graph_objects as go
import yfinance as yf

# 주식 데이터를 가져와 Plotly 차트로 변환하는 함수
def get_stock_chart(ticker):
    # 주식 데이터 다운로드 (최근 6개월)
    data, img = get_yfinance_stock_data(ticker)

    data = data.sort_values(by="Date")
    
    # Plotly 라인 차트 생성
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data["Date"], y=data["Close"], mode="lines", name="종가"))
    fig.add_trace
    fig.update_layout(
        title=f"{ticker.upper()} 주식 종가 라인 차트",
        xaxis_title="날짜",
        yaxis_title="가격",
        template="plotly_dark"  # 어두운 테마
    )

    return fig

# Gradio 인터페이스 설정
with gr.Blocks() as demo:
    gr.Markdown("# 주식 데이터 조회")
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder="예: AAPL")
    chart_output = gr.Plot(label="주식 차트")
    
    # Gradio에서 입력과 출력을 연결
    ticker_input.submit(get_stock_chart, inputs=ticker_input, outputs=chart_output)

# Gradio 앱 실행
demo.launch()

## 백엔드 정보 인기종목 등 연동

In [ ]:


# Gradio 인터페이스 설정
with gr.Blocks() as demo:
    gr.DataFrame(daily[0].head(10))

# Gradio 앱 실행
demo.launch()

## UI 개선 확인 버전

In [ ]:
# 백엔드 데이터 연동
from backend.stockchart import *

daily = dailyChart()
daily[0].head(10)
daily[1].head(10)

# 사용 가능한 티커 목록 가져오기
available_tickers = pd.read_csv('../data/available_tickers.csv')['tickers'].str.upper().tolist()
random.shuffle(available_tickers)

# 이미지 파일을 base64로 인코딩
with open("images/bg_01.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode()

with open("images/up_kr.jpeg", "rb") as image_file:
    encoded_up_image = base64.b64encode(image_file.read()).decode()

with open("images/down_kr.jpeg", "rb") as image_file:
    encoded_down_image = base64.b64encode(image_file.read()).decode()
    
# Gradio 인터페이스 설정
css_code = f"""
.gradio-container {{
    background-image: url(data:image/png;base64,{encoded_string});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 0.8;
}}

.custom-markdown h1, p {{
    color: white;
    text-align: center;
}}

div.custom-title > div > p {{
    color: white;
}}

div.small-font {{
    height: 170px !important;
    overflow-y: auto !important;
}}

div.small-font span {{
    font-size: 12px;
}}

.table-wrap {{
    max-height: 400px !important;
    overflow-y: auto !important;
}}

.predict_wrapper {{
    display: flex;
    justify-content: space-between;
}}

.predict_result {{
    width: 50%;
    height: 200px;
    background-color: white;
    border-radius: 10px;
    margin-left: 10px;
    padding: 10px;
}}

.predict_result_up {{
    width: 50%;
    height: 200px;
    border-radius: 10px;
    background-image: url(data:image/png;base64,{encoded_up_image});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 1;
}}

.predict_result_down {{
    width: 50%;
    height: 200px;
    border-radius: 10px;
    background-image: url(data:image/png;base64,{encoded_down_image});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 1;
}}

.predict_ticker, .predict_price, .start_price {{
    padding: 10px;
    font-size: 20px;
}}

span.price_down {{
    font-size: 3rem;
    color: blue;
    font-weight: 700;
}}

span.price_up {{
    font-size: 3rem;
    color: red;
    font-weight: 700;
}}
"""

import plotly.graph_objects as go

# 주식 데이터를 가져와 Plotly 차트로 변환하는 함수
def get_stock_chart(ticker):
    # 주식 데이터 다운로드 (최근 6개월)
    data, img = get_yfinance_stock_data(ticker)

    data = data.sort_values(by="Date")
    
    # Plotly 라인 차트 생성
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data["Date"], y=data["Close"], mode="lines", name="종가"))
    fig.add_trace
    fig.update_layout(
        title=f"{ticker.upper()} 주식 종가 라인 차트",
        xaxis_title="날짜",
        yaxis_title="가격",
        template="plotly_dark"  # 어두운 테마
    )

    return fig

# Gradio 인터페이스 설정
with gr.Blocks() as demo:
    gr.Markdown("# 주식 데이터 조회")
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder="예: AAPL")
    chart_output = gr.Plot(label="주식 차트")
    
    # Gradio에서 입력과 출력을 연결
    ticker_input.submit(get_stock_chart, inputs=ticker_input, outputs=chart_output)


with gr.Blocks(gr.themes.Monochrome(), css=css_code) as demo:
    gr.Markdown("# 천하제일 단타대회", elem_classes="custom-markdown")
    daily = dailyChart()
    volumn_top10 = daily[0].head(10)
    percent_top10 = daily[1].head(10)

    with gr.Row():
        gr.DataFrame(percent_top10, label="실시간 상승 주식", elem_classes="custom-title small-font")
        gr.DataFrame(volumn_top10, label="실시간 거래량 많은 주식", elem_classes="custom-title small-font")

    previous_tickers = gr.State(set())
    gr.Markdown("# 주식 가격 예측", elem_classes="custom-markdown")
    previous_tickers_display = gr.Radio(label="이전 조회된 티커", choices=[], interactive=True, visible=False)
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder=f"예: {", ".join(available_tickers[:5])} 등")
    warning_message = gr.Markdown(value="", visible=False)
    with gr.Row():
        submit_btn = gr.Button("조회")
        reset_btn = gr.Button("초기화")

    predict_result_up = gr.HTML(
        """
        <div class='predict_wrapper'>
            <div class='predict_result_up'>

            </div>
            <div class='predict_result'>
                {ticker}
            </div>
        </div>
        """,
        visible=False
    )

    predict_result_down = gr.HTML(
        """
        <div class='predict_wrapper'>
            <div class='predict_result_down'>

            </div>
            <div class='predict_result'>
                {ticker}
            </div>
        </div>
        """,
        visible=False
    )

    chart_output = gr.Image(label="주가 차트", elem_classes="custom-markdown", visible=False)
    data_output = gr.Dataframe(label="일별 주가 데이터", elem_classes="custom-title", visible=False)

    def update_visibility(ticker, previous_tickers):
        if not ticker:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="티커를 입력하세요.", visible=True, elem_classes="custom-markdown")

        ticker = ticker.upper()  # Convert ticker to uppercase
        if ticker.upper() not in available_tickers:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="입력하신 종목의 가격 예측은 추후에 제공될 예정입니다..", visible=True)

        data, img, isUp = predict_stock_price2(ticker)
        
        # 임시코드: AAPL이면 True, 그 외에는 False
        if ticker == "AAPL":
            isUp = True
        else:
            isUp = False

        previous_tickers.add(ticker)
        return (
            gr.update(visible=True, label=f"일별 주가 데이터 [{ticker}]", elem_classes="custom-title"),
            gr.update(visible=True),
            gr.update(visible=isUp, value=f"""
            <div class='predict_wrapper'>
                <div class='predict_result_up'>

                </div>
                <div class='predict_result'>
                    <div class='predict_ticker'>
                        Ticker: {ticker}
                    </div>  
                    <div class='predict_price'>
                        Predicted Price: <span class='price_up'>234.56(▲)</span>
                    </div>
                    <div class='start_price'>
                        Start Price : 231.45
                    </div>  
                </div>
            </div>
            """),
            gr.update(visible=not isUp, value=f"""
            <div class='predict_wrapper'>
                <div class='predict_result_down'>

                </div>
                <div class='predict_result'>
                    <div class='predict_ticker'>
                        Ticker: {ticker}
                    </div>  
                    <div class='predict_price'>
                        Predicted Price: <span class='price_down'>231.45(▼)</span>
                    </div>
                    <div class='start_price'>
                        Start Price : 234.56
                    </div>  
                </div>
            </div>
            """),
            data,
            img,
            gr.update(choices=list(previous_tickers), visible=True),
            gr.update(visible=False)
        )

    def reset_fields():
        return "", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)

    def update_ticker_input(selected_ticker):
        return selected_ticker, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(value="", visible=False)

    def auto_submit_ticker(selected_ticker, previous_tickers):
        return update_visibility(selected_ticker, previous_tickers)

    submit_btn.click(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output, previous_tickers_display, warning_message])
    ticker_input.submit(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output, previous_tickers_display, warning_message])
    reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message])
    previous_tickers_display.change(update_ticker_input, inputs=previous_tickers_display, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message])
    previous_tickers_display.change(auto_submit_ticker, inputs=[previous_tickers_display, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output, previous_tickers_display, warning_message])

# Gradio 앱 실행
demo.launch()
# demo.launch(share=True)

In [ ]:
# 백엔드 데이터 연동
from backend.stockchart import *

daily = dailyChart()
daily[0].head(10)
daily[1].head(10)

# 사용 가능한 티커 목록 가져오기
available_tickers = pd.read_csv('../data/available_tickers.csv')['tickers'].str.upper().tolist()
random.shuffle(available_tickers)

# 이미지 파일을 base64로 인코딩
with open("images/bg_01.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode()

with open("images/up_kr.jpeg", "rb") as image_file:
    encoded_up_image = base64.b64encode(image_file.read()).decode()

with open("images/down_kr.jpeg", "rb") as image_file:
    encoded_down_image = base64.b64encode(image_file.read()).decode()
    
# Gradio 인터페이스 설정
css_code = f"""
.gradio-container {{
    background-image: url(data:image/png;base64,{encoded_string});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 0.8;
}}

.custom-markdown h1, p {{
    color: white;
    text-align: center;
}}

div.custom-title > div > p {{
    color: white;
}}

div.small-font {{
    height: 170px !important;
    overflow-y: auto !important;
}}

div.small-font span {{
    font-size: 12px;
}}

.table-wrap {{
    max-height: 400px !important;
    overflow-y: auto !important;
}}

.predict_wrapper {{
    display: flex;
    justify-content: space-between;
}}

.predict_result {{
    width: 50%;
    height: 200px;
    background-color: white;
    border-radius: 10px;
    margin-left: 10px;
    padding: 10px;
}}

.predict_result_up {{
    width: 50%;
    height: 200px;
    border-radius: 10px;
    background-image: url(data:image/png;base64,{encoded_up_image});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 1;
}}

.predict_result_down {{
    width: 50%;
    height: 200px;
    border-radius: 10px;
    background-image: url(data:image/png;base64,{encoded_down_image});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 1;
}}

.predict_ticker, .predict_price, .start_price {{
    padding: 10px;
    font-size: 20px;
}}

span.price_down {{
    font-size: 3rem;
    color: blue;
    font-weight: 700;
}}

span.price_up {{
    font-size: 3rem;
    color: red;
    font-weight: 700;
}}
"""

import plotly.graph_objects as go

# 주식 데이터를 가져와 Plotly 차트로 변환하는 함수
def get_stock_chart(ticker):
    # 주식 데이터 다운로드 (최근 6개월)
    data, img = get_yfinance_stock_data(ticker)

    data = data.sort_values(by="Date")
    
    # Plotly 라인 차트 생성
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data["Date"], y=data["Close"], mode="lines", name="종가"))
    fig.add_trace
    fig.update_layout(
        title=f"{ticker.upper()} 주식 종가 라인 차트",
        xaxis_title="날짜",
        yaxis_title="가격",
        template="plotly_dark"  # 어두운 테마
    )

    return fig

# Gradio 인터페이스 설정
with gr.Blocks() as demo:
    gr.Markdown("# 주식 데이터 조회")
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder="예: AAPL")
    chart_output = gr.Plot(label="주식 차트")
    
    # Gradio에서 입력과 출력을 연결
    ticker_input.submit(get_stock_chart, inputs=ticker_input, outputs=chart_output)


with gr.Blocks(gr.themes.Monochrome(), css=css_code) as demo:
    gr.Markdown("# 천하제일 단타대회", elem_classes="custom-markdown")
    daily = dailyChart()
    volumn_top10 = daily[0].head(10)
    percent_top10 = daily[1].head(10)

    with gr.Row():
        gr.DataFrame(percent_top10, label="실시간 상승 주식", elem_classes="custom-title small-font")
        gr.DataFrame(volumn_top10, label="실시간 거래량 많은 주식", elem_classes="custom-title small-font")

    previous_tickers = gr.State(set())
    gr.Markdown("# 주식 가격 예측", elem_classes="custom-markdown")
    previous_tickers_display = gr.Radio(label="이전 조회된 티커", choices=[], interactive=True, visible=False)
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder=f"예: {", ".join(available_tickers[:5])} 등")
    warning_message = gr.Markdown(value="", visible=False)
    with gr.Row():
        submit_btn = gr.Button("조회")
        reset_btn = gr.Button("초기화")

    predict_result_up = gr.HTML(
        """
        <div class='predict_wrapper'>
            <div class='predict_result_up'>

            </div>
            <div class='predict_result'>
                {ticker}
            </div>
        </div>
        """,
        visible=False
    )

    predict_result_down = gr.HTML(
        """
        <div class='predict_wrapper'>
            <div class='predict_result_down'>

            </div>
            <div class='predict_result'>
                {ticker}
            </div>
        </div>
        """,
        visible=False
    )

    chart_output2 = gr.Plot(label="주식 차트")
    
    # Gradio에서 입력과 출력을 연결
    ticker_input.submit(get_stock_chart, inputs=ticker_input, outputs=chart_output2)

    chart_output = gr.Image(label="주가 차트", elem_classes="custom-markdown", visible=False)
    data_output = gr.Dataframe(label="일별 주가 데이터", elem_classes="custom-title", visible=False)

    def update_visibility(ticker, previous_tickers):
        if not ticker:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="티커를 입력하세요.", visible=True, elem_classes="custom-markdown")

        ticker = ticker.upper()  # Convert ticker to uppercase
        if ticker.upper() not in available_tickers:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="입력하신 종목의 가격 예측은 추후에 제공될 예정입니다..", visible=True)

        data, img, isUp = predict_stock_price2(ticker)
        
        # 임시코드: AAPL이면 True, 그 외에는 False
        if ticker == "AAPL":
            isUp = True
        else:
            isUp = False

        previous_tickers.add(ticker)
        return (
            gr.update(visible=True, label=f"일별 주가 데이터 [{ticker}]", elem_classes="custom-title"),
            gr.update(visible=True),
            gr.update(visible=isUp, value=f"""
            <div class='predict_wrapper'>
                <div class='predict_result_up'>

                </div>
                <div class='predict_result'>
                    <div class='predict_ticker'>
                        Ticker: {ticker}
                    </div>  
                    <div class='predict_price'>
                        Predicted Price: <span class='price_up'>234.56(▲)</span>
                    </div>
                    <div class='start_price'>
                        Start Price : 231.45
                    </div>  
                </div>
            </div>
            """),
            gr.update(visible=not isUp, value=f"""
            <div class='predict_wrapper'>
                <div class='predict_result_down'>

                </div>
                <div class='predict_result'>
                    <div class='predict_ticker'>
                        Ticker: {ticker}
                    </div>  
                    <div class='predict_price'>
                        Predicted Price: <span class='price_down'>231.45(▼)</span>
                    </div>
                    <div class='start_price'>
                        Start Price : 234.56
                    </div>  
                </div>
            </div>
            """),
            data,
            img,
            gr.update(choices=list(previous_tickers), visible=True),
            gr.update(visible=False)
        )

    def reset_fields():
        return "", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)

    def update_ticker_input(selected_ticker):
        return selected_ticker, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(value="", visible=False)

    def auto_submit_ticker(selected_ticker, previous_tickers):
        return update_visibility(selected_ticker, previous_tickers)
    
    submit_btn.click(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output, previous_tickers_display, warning_message])
    ticker_input.submit(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output, previous_tickers_display, warning_message])
    reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message])
    previous_tickers_display.change(update_ticker_input, inputs=previous_tickers_display, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message])
    previous_tickers_display.change(auto_submit_ticker, inputs=[previous_tickers_display, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output, previous_tickers_display, warning_message])

# Gradio 앱 실행
demo.launch()
# demo.launch(share=True)

In [ ]:
# 백엔드 데이터 연동
from backend.stockchart import *

daily = dailyChart()
daily[0].head(10)
daily[1].head(10)

# 사용 가능한 티커 목록 가져오기
available_tickers = pd.read_csv('../data/available_tickers.csv')['tickers'].str.upper().tolist()
random.shuffle(available_tickers)

# 이미지 파일을 base64로 인코딩
with open("images/bg_01.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode()

with open("images/up_kr.jpeg", "rb") as image_file:
    encoded_up_image = base64.b64encode(image_file.read()).decode()

with open("images/down_kr.jpeg", "rb") as image_file:
    encoded_down_image = base64.b64encode(image_file.read()).decode()
    
# Gradio 인터페이스 설정
css_code = f"""
.gradio-container {{
    background-image: url(data:image/png;base64,{encoded_string});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 0.8;
}}

.custom-markdown h1, p {{
    color: white;
    text-align: center;
}}

div.custom-title > div > p {{
    color: white;
}}

div.small-font {{
    height: 170px !important;
    overflow-y: auto !important;
}}

div.small-font span {{
    font-size: 12px;
}}

.table-wrap {{
    max-height: 400px !important;
    overflow-y: auto !important;
}}

.predict_wrapper {{
    display: flex;
    justify-content: space-between;
}}

.predict_result {{
    width: 50%;
    height: 200px;
    background-color: white;
    border-radius: 10px;
    margin-left: 10px;
    padding: 10px;
}}

.predict_result_up {{
    width: 50%;
    height: 200px;
    border-radius: 10px;
    background-image: url(data:image/png;base64,{encoded_up_image});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 1;
}}

.predict_result_down {{
    width: 50%;
    height: 200px;
    border-radius: 10px;
    background-image: url(data:image/png;base64,{encoded_down_image});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 1;
}}

.predict_ticker, .predict_price, .start_price {{
    padding: 10px;
    font-size: 20px;
}}

span.price_down {{
    font-size: 3rem;
    color: blue;
    font-weight: 700;
}}

span.price_up {{
    font-size: 3rem;
    color: red;
    font-weight: 700;
}}
"""

import plotly.graph_objects as go

# 주식 데이터를 가져와 Plotly 차트로 변환하는 함수
def get_stock_chart(ticker):
    # 주식 데이터 다운로드 (최근 6개월)
    data, img = get_yfinance_stock_data(ticker)

    data = data.sort_values(by="Date")
    
    # Plotly 라인 차트 생성
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data["Date"], y=data["Close"], mode="lines", name="종가"))
    fig.add_trace
    fig.update_layout(
        title=f"{ticker.upper()} 주식 종가 라인 차트",
        xaxis_title="날짜",
        yaxis_title="가격",
        template="plotly_dark"  # 어두운 테마
    )

    return fig

# Gradio 인터페이스 설정
with gr.Blocks() as demo:
    gr.Markdown("# 주식 데이터 조회")
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder="예: AAPL")
    chart_output = gr.Plot(label="주식 차트", visible=False)
    
    # Gradio에서 입력과 출력을 연결
    ticker_input.submit(get_stock_chart, inputs=ticker_input, outputs=chart_output)


with gr.Blocks(gr.themes.Monochrome(), css=css_code) as demo:
    gr.Markdown("# 천하제일 단타대회", elem_classes="custom-markdown")
    daily = dailyChart()
    volumn_top10 = daily[0].head(10)
    percent_top10 = daily[1].head(10)

    with gr.Row():
        gr.DataFrame(percent_top10, label="실시간 상승 주식", elem_classes="custom-title small-font")
        gr.DataFrame(volumn_top10, label="실시간 거래량 많은 주식", elem_classes="custom-title small-font")

    previous_tickers = gr.State(set())
    gr.Markdown("# 주식 가격 예측", elem_classes="custom-markdown")
    previous_tickers_display = gr.Radio(label="이전 조회된 티커", choices=[], interactive=True, visible=False)
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder=f"예: {", ".join(available_tickers[:5])} 등")
    warning_message = gr.Markdown(value="", visible=False)
    with gr.Row():
        submit_btn = gr.Button("조회")
        reset_btn = gr.Button("초기화")

    predict_result_up = gr.HTML(
        """
        <div class='predict_wrapper'>
            <div class='predict_result_up'>

            </div>
            <div class='predict_result'>
                {ticker}
            </div>
        </div>
        """,
        visible=False
    )

    predict_result_down = gr.HTML(
        """
        <div class='predict_wrapper'>
            <div class='predict_result_down'>

            </div>
            <div class='predict_result'>
                {ticker}
            </div>
        </div>
        """,
        visible=False
    )

    chart_output2 = gr.Plot(label="주식 차트", visible=False)
    
    # Gradio에서 입력과 출력을 연결
    ticker_input.submit(get_stock_chart, inputs=ticker_input, outputs=chart_output2)

    chart_output = gr.Image(label="주가 차트", elem_classes="custom-markdown", visible=False)
    data_output = gr.Dataframe(label="일별 주가 데이터", elem_classes="custom-title", visible=False)

    def update_visibility(ticker, previous_tickers):
        if not ticker:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="티커를 입력하세요.", visible=True, elem_classes="custom-markdown")

        ticker = ticker.upper()  # Convert ticker to uppercase
        if ticker.upper() not in available_tickers:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="입력하신 종목의 가격 예측은 추후에 제공될 예정입니다..", visible=True)

        data, img, isUp = predict_stock_price2(ticker)
        
        # 임시코드: AAPL이면 True, 그 외에는 False
        if ticker == "AAPL":
            isUp = True
        else:
            isUp = False

        previous_tickers.add(ticker)
        return (
            gr.update(visible=True, label=f"일별 주가 데이터 [{ticker}]", elem_classes="custom-title"),
            gr.update(visible=False),
            gr.update(visible=isUp, value=f"""
            <div class='predict_wrapper'>
                <div class='predict_result_up'>

                </div>
                <div class='predict_result'>
                    <div class='predict_ticker'>
                        Ticker: {ticker}
                    </div>  
                    <div class='predict_price'>
                        Predicted Price: <span class='price_up'>234.56(▲)</span>
                    </div>
                    <div class='start_price'>
                        Start Price : 231.45
                    </div>  
                </div>
            </div>
            """),
            gr.update(visible=not isUp, value=f"""
            <div class='predict_wrapper'>
                <div class='predict_result_down'>

                </div>
                <div class='predict_result'>
                    <div class='predict_ticker'>
                        Ticker: {ticker}
                    </div>  
                    <div class='predict_price'>
                        Predicted Price: <span class='price_down'>231.45(▼)</span>
                    </div>
                    <div class='start_price'>
                        Start Price : 234.56
                    </div>  
                </div>
            </div>
            """),
            data,
            gr.update(visible=True),
            gr.update(choices=list(previous_tickers), visible=True),
            gr.update(visible=False)
        )

    def reset_fields():
        return "", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)

    def update_ticker_input(selected_ticker):
        return selected_ticker, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(value="", visible=False)

    def auto_submit_ticker(selected_ticker, previous_tickers):
        return update_visibility(selected_ticker, previous_tickers)
    
    submit_btn.click(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])
    ticker_input.submit(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])
    reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message])
    previous_tickers_display.change(update_ticker_input, inputs=previous_tickers_display, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message])
    previous_tickers_display.change(auto_submit_ticker, inputs=[previous_tickers_display, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])

# Gradio 앱 실행
demo.launch()
# demo.launch(share=True)

In [ ]:
# 백엔드 데이터 연동
from backend.stockchart import *
from gradio_modal import Modal

daily = dailyChart()
daily[0].head(10)
daily[1].head(10)

# 사용 가능한 티커 목록 가져오기
available_tickers = pd.read_csv('../data/available_tickers.csv')['tickers'].str.upper().tolist()
random.shuffle(available_tickers)

# 이미지 파일을 base64로 인코딩
with open("images/bg_01.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode()

with open("images/up_kr.jpeg", "rb") as image_file:
    encoded_up_image = base64.b64encode(image_file.read()).decode()

with open("images/down_kr.jpeg", "rb") as image_file:
    encoded_down_image = base64.b64encode(image_file.read()).decode()
    
# Gradio 인터페이스 설정
css_code = f"""
.gradio-container {{
    background-image: url(data:image/png;base64,{encoded_string});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 0.8;
}}

.custom-markdown h1, p {{
    color: white;
    text-align: center;
}}

div.custom-title > div > p {{
    color: white;
}}

div.small-font {{
    height: 170px !important;
    overflow-y: auto !important;
}}

div.small-font span {{
    font-size: 12px;
}}

.table-wrap {{
    max-height: 400px !important;
    overflow-y: auto !important;
}}

.predict_wrapper {{
    display: flex;
    justify-content: space-between;
}}

.predict_result {{
    width: 50%;
    height: 200px;
    background-color: white;
    border-radius: 10px;
    margin-left: 10px;
    padding: 10px;
}}

.predict_result_up {{
    width: 50%;
    height: 200px;
    border-radius: 10px;
    background-image: url(data:image/png;base64,{encoded_up_image});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 1;
}}

.predict_result_down {{
    width: 50%;
    height: 200px;
    border-radius: 10px;
    background-image: url(data:image/png;base64,{encoded_down_image});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 1;
}}

.predict_ticker, .predict_price, .start_price {{
    padding: 10px;
    font-size: 20px;
}}

span.price_down {{
    font-size: 3rem;
    color: blue;
    font-weight: 700;
}}

span.price_up {{
    font-size: 3rem;
    color: red;
    font-weight: 700;
}}
"""

import plotly.graph_objects as go

# 주식 데이터를 가져와 Plotly 차트로 변환하는 함수
def get_stock_chart(ticker):
    # 주식 데이터 다운로드 (최근 6개월)
    data, img = get_yfinance_stock_data(ticker)

    data = data.sort_values(by="Date")
    
    # Plotly 라인 차트 생성
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data["Date"], y=data["Close"], mode="lines", name="종가"))
    fig.add_trace
    fig.update_layout(
        title=f"{ticker.upper()} 주식 종가 라인 차트",
        xaxis_title="날짜",
        yaxis_title="가격",
        template="plotly_dark"  # 어두운 테마
    )

    return fig


def close_modal():
    return gr.update(visible=False)

with gr.Blocks(gr.themes.Monochrome(), css=css_code) as demo:

    with Modal(visible=True,allow_user_close=False) as modal:
            gr.Image("images/start.png")
            agree_btn = gr.Button("START")

    agree_btn.click(close_modal, None, modal)

    gr.Markdown("# 천하제일 단타대회", elem_classes="custom-markdown")
    daily = dailyChart()
    volumn_top10 = daily[0].head(10)
    percent_top10 = daily[1].head(10)

    with gr.Row():
        gr.DataFrame(percent_top10, label="실시간 상승 주식", elem_classes="custom-title small-font")
        gr.DataFrame(volumn_top10, label="실시간 거래량 많은 주식", elem_classes="custom-title small-font")

    previous_tickers = gr.State(set())
    gr.Markdown("# 주식 가격 예측", elem_classes="custom-markdown")
    previous_tickers_display = gr.Radio(label="이전 조회된 티커", choices=[], interactive=True, visible=False)
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder=f"예: {", ".join(available_tickers[:5])} 등")
    warning_message = gr.Markdown(value="", visible=False)
    with gr.Row():
        submit_btn = gr.Button("조회")
        reset_btn = gr.Button("초기화")

    predict_result_up = gr.HTML(
        """
        <div class='predict_wrapper'>
            <div class='predict_result_up'>

            </div>
            <div class='predict_result'>
                {ticker}
            </div>
        </div>
        """,
        visible=False
    )

    predict_result_down = gr.HTML(
        """
        <div class='predict_wrapper'>
            <div class='predict_result_down'>

            </div>
            <div class='predict_result'>
                {ticker}
            </div>
        </div>
        """,
        visible=False
    )

    chart_output2 = gr.Plot(label="주식 차트", visible=False)
    
    # Gradio에서 입력과 출력을 연결
    ticker_input.submit(get_stock_chart, inputs=ticker_input, outputs=chart_output2)

    chart_output = gr.Image(label="주가 차트", elem_classes="custom-markdown", visible=False)
    data_output = gr.Dataframe(label="일별 주가 데이터", elem_classes="custom-title", visible=False)

    def update_visibility(ticker, previous_tickers):
        if not ticker:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="티커를 입력하세요.", visible=True, elem_classes="custom-markdown")

        ticker = ticker.upper()  # Convert ticker to uppercase
        if ticker.upper() not in available_tickers:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="입력하신 종목의 가격 예측은 추후에 제공될 예정입니다..", visible=True)

        data, img, isUp = predict_stock_price2(ticker)
        
        # 임시코드: AAPL이면 True, 그 외에는 False
        if ticker == "AAPL":
            isUp = True
        else:
            isUp = False

        previous_tickers.add(ticker)
        return (
            gr.update(visible=True, label=f"일별 주가 데이터 [{ticker}]", elem_classes="custom-title"),
            gr.update(visible=False),
            gr.update(visible=isUp, value=f"""
            <div class='predict_wrapper'>
                <div class='predict_result_up'>

                </div>
                <div class='predict_result'>
                    <div class='predict_ticker'>
                        Ticker: {ticker}
                    </div>  
                    <div class='predict_price'>
                        Predicted Price: <span class='price_up'>234.56(▲)</span>
                    </div>
                    <div class='start_price'>
                        Start Price : 231.45
                    </div>  
                </div>
            </div>
            """),
            gr.update(visible=not isUp, value=f"""
            <div class='predict_wrapper'>
                <div class='predict_result_down'>

                </div>
                <div class='predict_result'>
                    <div class='predict_ticker'>
                        Ticker: {ticker}
                    </div>  
                    <div class='predict_price'>
                        Predicted Price: <span class='price_down'>231.45(▼)</span>
                    </div>
                    <div class='start_price'>
                        Start Price : 234.56
                    </div>  
                </div>
            </div>
            """),
            data,
            gr.update(visible=True),
            gr.update(choices=list(previous_tickers), visible=True),
            gr.update(visible=False)
        )

    def reset_fields():
        return "", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)

    def update_ticker_input(selected_ticker):
        return selected_ticker, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(value="", visible=False)

    def auto_submit_ticker(selected_ticker, previous_tickers):
        return update_visibility(selected_ticker, previous_tickers)
    
    submit_btn.click(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])
    ticker_input.submit(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])
    reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message])
    previous_tickers_display.change(update_ticker_input, inputs=previous_tickers_display, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message])
    previous_tickers_display.change(auto_submit_ticker, inputs=[previous_tickers_display, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])

# Gradio 앱 실행
demo.launch()
# demo.launch(share=True)

## 기업정보 및 인공지능 윤리

In [ ]:
# 백엔드 데이터 연동
from backend.stockchart import *
from gradio_modal import Modal

daily = dailyChart()
daily[0].head(10)
daily[1].head(10)

# 사용 가능한 티커 목록 가져오기
available_tickers = pd.read_csv('../data/available_tickers.csv')['tickers'].str.upper().tolist()
random.shuffle(available_tickers)

# 이미지 파일을 base64로 인코딩
with open("images/bg_01.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode()

with open("images/up_kr.jpeg", "rb") as image_file:
    encoded_up_image = base64.b64encode(image_file.read()).decode()

with open("images/down_kr.jpeg", "rb") as image_file:
    encoded_down_image = base64.b64encode(image_file.read()).decode()
    
# Gradio 인터페이스 설정
css_code = f"""
.gradio-container {{
    background-image: url(data:image/png;base64,{encoded_string});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 0.8;
}}

.custom-markdown h1, p {{
    color: white;
    text-align: center;
}}

div.custom-title > div > p {{
    color: white;
}}

div.small-font {{
    height: 170px !important;
    overflow-y: auto !important;
}}

div.small-font span {{
    font-size: 12px;
}}

.table-wrap {{
    max-height: 400px !important;
    overflow-y: auto !important;
}}

.predict_wrapper {{
    display: flex;
    justify-content: space-between;
}}

.predict_result {{
    width: 50%;
    height: 200px;
    background-color: white;
    border-radius: 10px;
    margin-left: 10px;
    padding: 10px;
}}

.predict_result_up {{
    width: 50%;
    height: 200px;
    border-radius: 10px;
    background-image: url(data:image/png;base64,{encoded_up_image});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 1;
}}

.predict_result_down {{
    width: 50%;
    height: 200px;
    border-radius: 10px;
    background-image: url(data:image/png;base64,{encoded_down_image});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 1;
}}

.predict_ticker, .predict_price, .start_price {{
    padding: 10px;
    font-size: 20px;
}}

span.price_down {{
    font-size: 3rem;
    color: blue;
    font-weight: 700;
}}

span.price_up {{
    font-size: 3rem;
    color: red;
    font-weight: 700;
}}
"""

import plotly.graph_objects as go

# 주식 데이터를 가져와 Plotly 차트로 변환하는 함수
def get_stock_chart(ticker):
    # 주식 데이터 다운로드 (최근 6개월)
    data, img = get_yfinance_stock_data(ticker)

    data = data.sort_values(by="Date")
    
    # Plotly 라인 차트 생성
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data["Date"], y=data["Close"], mode="lines", name="종가"))
    fig.add_trace
    fig.update_layout(
        title=f"{ticker.upper()} 주식 종가 라인 차트",
        xaxis_title="날짜",
        yaxis_title="가격",
        template="plotly_dark"  # 어두운 테마
    )

    return fig


def close_modal():
    return gr.update(visible=False)

with gr.Blocks(gr.themes.Monochrome(), css=css_code) as demo:

    with Modal(visible=True,allow_user_close=False) as modal:
            gr.Image("images/start.png")
            agree_btn = gr.Button("START")

    agree_btn.click(close_modal, None, modal)

    gr.Markdown("# 천하제일 단타대회", elem_classes="custom-markdown")
    daily = dailyChart()
    volumn_top10 = daily[0].head(10)
    percent_top10 = daily[1].head(10)

    with gr.Row():
        gr.DataFrame(percent_top10, label="실시간 상승 주식", elem_classes="custom-title small-font")
        gr.DataFrame(volumn_top10, label="실시간 거래량 많은 주식", elem_classes="custom-title small-font")

    previous_tickers = gr.State(set())
    gr.Markdown("# 주식 가격 예측", elem_classes="custom-markdown")
    previous_tickers_display = gr.Radio(label="이전 조회된 티커", choices=[], interactive=True, visible=False)
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder=f"예: {", ".join(available_tickers[:5])} 등")
    warning_message = gr.Markdown(value="", visible=False)
    with gr.Row():
        submit_btn = gr.Button("조회")
        reset_btn = gr.Button("초기화")

    predict_result_up = gr.HTML(
        """
        <div class='predict_wrapper'>
            <div class='predict_result_up'>

            </div>
            <div class='predict_result'>
                {ticker}
            </div>
        </div>
        """,
        visible=False
    )

    predict_result_down = gr.HTML(
        """
        <div class='predict_wrapper'>
            <div class='predict_result_down'>

            </div>
            <div class='predict_result'>
                {ticker}
            </div>
        </div>
        """,
        visible=False
    )

    chart_output2 = gr.Plot(label="주식 차트", visible=False)
    
    # Gradio에서 입력과 출력을 연결
    ticker_input.submit(get_stock_chart, inputs=ticker_input, outputs=chart_output2)

    chart_output = gr.Image(label="주가 차트", elem_classes="custom-markdown", visible=False)
    data_output = gr.Dataframe(label="일별 주가 데이터", elem_classes="custom-title", visible=False)

    def update_visibility(ticker, previous_tickers):
        if not ticker:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="티커를 입력하세요.", visible=True, elem_classes="custom-markdown")

        ticker = ticker.upper()  # Convert ticker to uppercase
        if ticker.upper() not in available_tickers:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="입력하신 종목의 가격 예측은 추후에 제공될 예정입니다..", visible=True)

        data, img, isUp = predict_stock_price2(ticker)
        
        # 임시코드: AAPL이면 True, 그 외에는 False
        if ticker == "AAPL":
            isUp = True
        else:
            isUp = False

        previous_tickers.add(ticker)
        return (
            gr.update(visible=True, label=f"일별 주가 데이터 [{ticker}]", elem_classes="custom-title"),
            gr.update(visible=False),
            gr.update(visible=isUp, value=f"""
            <div class='predict_wrapper'>
                <div class='predict_result_up'>

                </div>
                <div class='predict_result'>
                    <div class='predict_ticker'>
                        Ticker: {ticker}
                    </div>  
                    <div class='predict_price'>
                        Predicted Price: <span class='price_up'>234.56(▲)</span>
                    </div>
                    <div class='start_price'>
                        Start Price : 231.45
                    </div>  
                </div>
            </div>
            """),
            gr.update(visible=not isUp, value=f"""
            <div class='predict_wrapper'>
                <div class='predict_result_down'>

                </div>
                <div class='predict_result'>
                    <div class='predict_ticker'>
                        Ticker: {ticker}
                    </div>  
                    <div class='predict_price'>
                        Predicted Price: <span class='price_down'>231.45(▼)</span>
                    </div>
                    <div class='start_price'>
                        Start Price : 234.56
                    </div>  
                </div>
            </div>
            """),
            data,
            gr.update(visible=True),
            gr.update(choices=list(previous_tickers), visible=True),
            gr.update(visible=False)
        )

    # def reset_fields():
    #     return "", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)

    def reset_fields():
        return "", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)


    def update_ticker_input(selected_ticker):
        return selected_ticker, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(value="", visible=False)

    def auto_submit_ticker(selected_ticker, previous_tickers):
        return update_visibility(selected_ticker, previous_tickers)
    
    submit_btn.click(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])
    ticker_input.submit(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])
    # reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message])
    reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message, chart_output2])
    previous_tickers_display.change(update_ticker_input, inputs=previous_tickers_display, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message])
    previous_tickers_display.change(auto_submit_ticker, inputs=[previous_tickers_display, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])

# Gradio 앱 실행
demo.launch()
# demo.launch(share=True)

## 머신러닝 연동

In [7]:
# 백엔드 데이터 연동
from backend.stockchart import *
from backend.exchangerate import *
from gradio_modal import Modal

daily = dailyChart()
daily[0].head(10)
daily[1].head(10)

# 사용 가능한 티커 목록 가져오기
available_tickers = pd.read_csv('../data/available_tickers.csv')['tickers'].str.upper().tolist()
random.shuffle(available_tickers)

# 이미지 파일을 base64로 인코딩
with open("images/bg_01.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode()

with open("images/up_kr.jpeg", "rb") as image_file:
    encoded_up_image = base64.b64encode(image_file.read()).decode()

with open("images/down_kr.jpeg", "rb") as image_file:
    encoded_down_image = base64.b64encode(image_file.read()).decode()
    
# Gradio 인터페이스 설정
css_code = f"""
.gradio-container {{
    background-image: url(data:image/png;base64,{encoded_string});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 0.8;
}}

.custom-markdown h1, p {{
    color: white;
    text-align: center;
}}

div.custom-title > div > p {{
    color: white;
}}

div.small-font {{
    height: 170px !important;
    overflow-y: auto !important;
}}

div.small-font span {{
    font-size: 12px;
}}

.table-wrap {{
    max-height: 400px !important;
    overflow-y: auto !important;
}}

.predict_wrapper {{
    display: flex;
    justify-content: space-between;
}}

.predict_result {{
    width: 50%;
    height: 200px;
    background-color: white;
    border-radius: 10px;
    margin-left: 10px;
    padding: 10px;
}}

.predict_result_up {{
    width: 50%;
    height: 200px;
    border-radius: 10px;
    background-image: url(data:image/png;base64,{encoded_up_image});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 1;
}}

.predict_result_down {{
    width: 50%;
    height: 200px;
    border-radius: 10px;
    background-image: url(data:image/png;base64,{encoded_down_image});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 1;
}}

.predict_ticker, .predict_price, .start_price {{
    padding: 10px;
    font-size: 20px;
}}

span.price_down {{
    font-size: 3rem;
    color: blue;
    font-weight: 700;
}}

span.price_up {{
    font-size: 3rem;
    color: red;
    font-weight: 700;
}}
"""

import requests

# 요청 URL
url = "http://121.162.37.155:5000/predict"

# 요청 헤더 설정
headers = {
    "Content-Type": "application/json"
}

# 요청 본문 설정 (필요한 데이터로 수정)
payload = {
    # API에서 요구하는 데이터를 여기에 추가하세요.
    "ticker": "AAPL"
}

try:
    # POST 요청 보내기
    response = requests.get(url, json=payload, headers=headers)
    
    # 요청이 성공했는지 확인
    response.raise_for_status()
    
    # JSON 응답 받기
    json_response = response.json()
    print("응답 받은 JSON:", json_response)
    
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP 에러 발생: {http_err}")
except Exception as err:
    print(f"기타 에러 발생: {err}")


import plotly.graph_objects as go

# 주식 데이터를 가져와 Plotly 차트로 변환하는 함수
def get_stock_chart(ticker):
    # 주식 데이터 다운로드 (최근 6개월)
    data, img = get_yfinance_stock_data(ticker)

    data = data.sort_values(by="Date")
    
    # Plotly 라인 차트 생성
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data["Date"], y=data["Close"], mode="lines", name="종가"))
    fig.add_trace
    fig.update_layout(
        title=f"{ticker.upper()} 주식 종가 라인 차트",
        xaxis_title="날짜",
        yaxis_title="가격",
        template="plotly_white"  # 밝은 테마
    )

    return fig


def close_modal():
    return gr.update(visible=False)

with gr.Blocks(gr.themes.Monochrome(), css=css_code) as demo:

    with Modal(visible=True,allow_user_close=False) as modal:
            gr.Image("images/start.png")
            agree_btn = gr.Button("START")

    agree_btn.click(close_modal, None, modal)

    exchange_rate = exchangerate()

    with gr.Row():
        modal_button = gr.Button("기업정보")
        gr.Markdown("# 천하제일 단타대회", elem_classes="custom-markdown")
        gr.Markdown(f"1 USD: {exchange_rate}", elem_classes="custom-markdown")
        
    modal_button.click(lambda: modal.show(), None, None)

    daily = dailyChart()
    volumn_top10 = daily[0].head(10)
    percent_top10 = daily[1].head(10)

    with gr.Row():
        gr.DataFrame(percent_top10, label="실시간 상승 주식", elem_classes="custom-title small-font")
        gr.DataFrame(volumn_top10, label="실시간 거래량 많은 주식", elem_classes="custom-title small-font")

    previous_tickers = gr.State(set())
    gr.Markdown("# 주식 가격 예측", elem_classes="custom-markdown")
    previous_tickers_display = gr.Radio(label="이전 조회된 티커", choices=[], interactive=True, visible=False)
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder=f"예: {", ".join(available_tickers[:5])} 등")
    warning_message = gr.Markdown(value="", visible=False)
    with gr.Row():
        submit_btn = gr.Button("조회")
        reset_btn = gr.Button("초기화")

    predict_result_up = gr.HTML(
        """
        <div class='predict_wrapper'>
            <div class='predict_result_up'>

            </div>
            <div class='predict_result'>
                {ticker}
            </div>
        </div>
        """,
        visible=False
    )

    predict_result_down = gr.HTML(
        """
        <div class='predict_wrapper'>
            <div class='predict_result_down'>

            </div>
            <div class='predict_result'>
                {ticker}
            </div>
        </div>
        """,
        visible=False
    )

    chart_output2 = gr.Plot(label="주식 차트", visible=False)
    
    # Gradio에서 입력과 출력을 연결
    ticker_input.submit(get_stock_chart, inputs=ticker_input, outputs=chart_output2)

    chart_output = gr.Image(label="주가 차트", elem_classes="custom-markdown", visible=False)
    data_output = gr.Dataframe(label="일별 주가 데이터", elem_classes="custom-title", visible=False)

    def update_visibility(ticker, previous_tickers):
        if not ticker:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="티커를 입력하세요.", visible=True, elem_classes="custom-markdown")

        ticker = ticker.upper()  # Convert ticker to uppercase
        if ticker.upper() not in available_tickers:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="입력하신 종목의 가격 예측은 추후에 제공될 예정입니다..", visible=True)

        data, img, isUp, open, close, prediction = predict_stock_price3(ticker)
        open = round(open, 3)
        close = round(close, 3)

        # 임시코드: AAPL이면 True, 그 외에는 False
        # if ticker == "AAPL":
        #     isUp = True
        # else:
        #     isUp = False

        previous_tickers.add(ticker)
        return (
            gr.update(visible=True, label=f"일별 주가 데이터 [{ticker}]", elem_classes="custom-title"),
            gr.update(visible=False),
            gr.update(visible=isUp, value=f"""
            <div class='predict_wrapper'>
                <div class='predict_result_up'>

                </div>
                <div class='predict_result'>
                    <div class='predict_ticker'>
                        Ticker: {ticker}
                    </div>  
                    <div class='predict_price'>
                        Predicted Price: <span class='price_up'>{close}(▲)</span>
                    </div>
                    <div class='start_price'>
                        Open Price : {open}
                    </div>  
                </div>
            </div>
            """),
            gr.update(visible=not isUp, value=f"""
            <div class='predict_wrapper'>
                <div class='predict_result_down'>

                </div>
                <div class='predict_result'>
                    <div class='predict_ticker'>
                        Ticker: {ticker}
                    </div>  
                    <div class='predict_price'>
                        Predicted Price: <span class='price_down'>{close}(▼)</span>
                    </div>
                    <div class='start_price'>
                        Open Price : {open}
                    </div>  
                </div>
            </div>
            """),
            data,
            gr.update(visible=True),
            gr.update(choices=list(previous_tickers), visible=True),
            gr.update(visible=False)
        )

    # def reset_fields():
    #     return "", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)

    def reset_fields():
        return "", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)


    def update_ticker_input(selected_ticker):
        return selected_ticker, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(value="", visible=False)

    def auto_submit_ticker(selected_ticker, previous_tickers):
        return update_visibility(selected_ticker, previous_tickers)
    
    submit_btn.click(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])
    ticker_input.submit(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])
    # reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message])
    reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message, chart_output2])
    previous_tickers_display.change(update_ticker_input, inputs=previous_tickers_display, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message])
    previous_tickers_display.change(auto_submit_ticker, inputs=[previous_tickers_display, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])

# Gradio 앱 실행
demo.launch()
# demo.launch(share=True)

HTTP 에러 발생: 405 Client Error: METHOD NOT ALLOWED for url: http://121.162.37.155:5000/predict
* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## 분석 관련 정보 출력

In [6]:
# 백엔드 데이터 연동
from backend.stockchart import *
from backend.exchangerate import *
from gradio_modal import Modal

daily = dailyChart()
daily[0].head(10)
daily[1].head(10)

# 사용 가능한 티커 목록 가져오기
available_tickers = pd.read_csv('../data/available_tickers.csv')['tickers'].str.upper().tolist()
random.shuffle(available_tickers)

# 이미지 파일을 base64로 인코딩
with open("images/bg_01.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode()

with open("images/up_kr.jpeg", "rb") as image_file:
    encoded_up_image = base64.b64encode(image_file.read()).decode()

with open("images/down_kr.jpeg", "rb") as image_file:
    encoded_down_image = base64.b64encode(image_file.read()).decode()
    
# Gradio 인터페이스 설정
css_code = f"""
.gradio-container {{
    background-image: url(data:image/png;base64,{encoded_string});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 0.8;
}}

.custom-markdown h1, p {{
    color: white;
    text-align: center;
}}

div.custom-title > div > p {{
    color: white;
}}

div.small-font {{
    height: 170px !important;
    overflow-y: auto !important;
}}

div.small-font span {{
    font-size: 12px;
}}

.table-wrap {{
    max-height: 400px !important;
    overflow-y: auto !important;
}}

.predict_wrapper {{
    display: flex;
    justify-content: space-between;
}}

.predict_result {{
    width: 50%;
    height: 200px;
    background-color: white;
    border-radius: 10px;
    margin-left: 10px;
    padding: 10px;
}}

.predict_result_up {{
    width: 50%;
    height: 200px;
    border-radius: 10px;
    background-image: url(data:image/png;base64,{encoded_up_image});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 1;
}}

.predict_result_down {{
    width: 50%;
    height: 200px;
    border-radius: 10px;
    background-image: url(data:image/png;base64,{encoded_down_image});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 1;
}}

.predict_ticker, .predict_price, .start_price {{
    padding: 10px;
    font-size: 20px;
}}

span.price_down {{
    font-size: 3rem;
    color: blue;
    font-weight: 700;
}}

span.price_up {{
    font-size: 3rem;
    color: red;
    font-weight: 700;
}}
"""

import requests

# 요청 URL
url = "http://121.162.37.155:5000/predict"

# 요청 헤더 설정
headers = {
    "Content-Type": "application/json"
}

# 요청 본문 설정 (필요한 데이터로 수정)
payload = {
    # API에서 요구하는 데이터를 여기에 추가하세요.
    "ticker": "AAPL"
}

try:
    # POST 요청 보내기
    response = requests.get(url, json=payload, headers=headers)
    
    # 요청이 성공했는지 확인
    response.raise_for_status()
    
    # JSON 응답 받기
    json_response = response.json()
    print("응답 받은 JSON:", json_response)
    
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP 에러 발생: {http_err}")
except Exception as err:
    print(f"기타 에러 발생: {err}")


import plotly.graph_objects as go

# 주식 데이터를 가져와 Plotly 차트로 변환하는 함수
def get_stock_chart(ticker):
    # 주식 데이터 다운로드 (최근 6개월)
    data, img = get_yfinance_stock_data(ticker)

    data = data.sort_values(by="Date")
    
    # Plotly 라인 차트 생성
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data["Date"], y=data["Close"], mode="lines", name="종가"))
    fig.add_trace
    fig.update_layout(
        title=f"{ticker.upper()} 주식 종가 라인 차트",
        xaxis_title="날짜",
        yaxis_title="가격",
        template="plotly_white"  # 밝은 테마
    )

    return fig

def info_fn():
    information="""
    [오른쪽 X를 눌러 창 닫기]<br>
    < 분석 기준 ><br>
    - open(시작가)<br>
    - close(종가)<br>
    - InterestRate(금리)<br>
    - ExchangeRate(1유로 기준 달러 환율)<br>
    - VIX(변동성 지수(=공포 지수))<br>
    - TEDSpread(은행 간 대출 금리와 단기 미국 국채 금리 간의 차이)<br>
    - EFFR(예금 기관들이 서로에게 하룻밤 동안 예비 잔고를 대출할때 적용되는 금리)<br>
    - Gold(금 가격)<br>
    - Oil(유가)<br>
    - Close_InterestRate_Corr(종가와 이자율의 상관계수)<br>
    - Close_VIX_Corr(종가-공포지수 상관계수)<br>
    - Daily_Return(일일 수익률)<br>
    - Volatility(30일 이동평균편차)<br>
    - Rolling_Mean_Close(30일 이동평균)<br><br>

    < 분석 기준 출처 ><br>
    - open,close,Gold, Oil = YFinance<br>
    - InterestRate, ExchangeRate, VIX,<br> TEDSpread, EFFR = FRED<br>
    - 달러 환율=네이버 은행고시환율(하나은행)<br><br>

    < 분석 기간 ><br>
    2010-01-01 ~ 2024-11-06<br><br>

    < 분석 모델 ><br> 
    RandomForestRegression<br><br>
    """
    gr.Info(information,duration=None,title="")

def close_modal():
    return gr.update(visible=False)

with gr.Blocks(gr.themes.Monochrome(), css=css_code) as demo:

    with Modal(visible=True,allow_user_close=False) as modal:
            gr.Image("images/start.png")
            agree_btn = gr.Button("START")

    agree_btn.click(close_modal, None, modal)



    exchange_rate = exchangerate()

    with gr.Row():
        trigger_info = gr.Button(value="분석 관련 정보")
        gr.Markdown("# 천하제일 단타대회", elem_classes="custom-markdown")
        gr.Markdown(f"1 USD: {exchange_rate}", elem_classes="custom-markdown")
        
    trigger_info.click(info_fn, None, None)
    modal_button.click(lambda: modal.show(), None, None)

    daily = dailyChart()
    volumn_top10 = daily[0].head(10)
    percent_top10 = daily[1].head(10)

    with gr.Row():
        gr.DataFrame(percent_top10, label="실시간 상승 주식", elem_classes="custom-title small-font")
        gr.DataFrame(volumn_top10, label="실시간 거래량 많은 주식", elem_classes="custom-title small-font")

    previous_tickers = gr.State(set())
    gr.Markdown("# 주식 가격 예측", elem_classes="custom-markdown")
    previous_tickers_display = gr.Radio(label="이전 조회된 티커", choices=[], interactive=True, visible=False)
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder=f"예: {", ".join(available_tickers[:5])} 등")
    warning_message = gr.Markdown(value="", visible=False)
    with gr.Row():
        submit_btn = gr.Button("조회")
        reset_btn = gr.Button("초기화")

    predict_result_up = gr.HTML(
        """
        <div class='predict_wrapper'>
            <div class='predict_result_up'>

            </div>
            <div class='predict_result'>
                {ticker}
            </div>
        </div>
        """,
        visible=False
    )

    predict_result_down = gr.HTML(
        """
        <div class='predict_wrapper'>
            <div class='predict_result_down'>

            </div>
            <div class='predict_result'>
                {ticker}
            </div>
        </div>
        """,
        visible=False
    )

    chart_output2 = gr.Plot(label="주식 차트", visible=False)
    
    # Gradio에서 입력과 출력을 연결
    ticker_input.submit(get_stock_chart, inputs=ticker_input, outputs=chart_output2)
    previous_tickers_display.change(update_ticker_input, inputs=previous_tickers_display, outputs=chart_output2)

    chart_output = gr.Image(label="주가 차트", elem_classes="custom-markdown", visible=False)
    data_output = gr.Dataframe(label="일별 주가 데이터", elem_classes="custom-title", visible=False)

    def update_visibility(ticker, previous_tickers):
        if not ticker:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="티커를 입력하세요.", visible=True, elem_classes="custom-markdown")

        ticker = ticker.upper()  # Convert ticker to uppercase
        if ticker.upper() not in available_tickers:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="입력하신 종목의 가격 예측은 추후에 제공될 예정입니다..", visible=True)

        data, img, isUp, open, close, prediction = predict_stock_price3(ticker)
        open = round(open, 3)
        close = round(close, 3)

        # 임시코드: AAPL이면 True, 그 외에는 False
        # if ticker == "AAPL":
        #     isUp = True
        # else:
        #     isUp = False

        previous_tickers.add(ticker)
        return (
            gr.update(visible=True, label=f"일별 주가 데이터 [{ticker}]", elem_classes="custom-title"),
            gr.update(visible=False),
            gr.update(visible=isUp, value=f"""
            <div class='predict_wrapper'>
                <div class='predict_result_up'>

                </div>
                <div class='predict_result'>
                    <div class='predict_ticker'>
                        Ticker: {ticker}
                    </div>  
                    <div class='predict_price'>
                        Predicted Price: <span class='price_up'>{close}(▲)</span>
                    </div>
                    <div class='start_price'>
                        Open Price : {open}
                    </div>  
                </div>
            </div>
            """),
            gr.update(visible=not isUp, value=f"""
            <div class='predict_wrapper'>
                <div class='predict_result_down'>

                </div>
                <div class='predict_result'>
                    <div class='predict_ticker'>
                        Ticker: {ticker}
                    </div>  
                    <div class='predict_price'>
                        Predicted Price: <span class='price_down'>{close}(▼)</span>
                    </div>
                    <div class='start_price'>
                        Open Price : {open}
                    </div>  
                </div>
            </div>
            """),
            data,
            gr.update(visible=True),
            gr.update(choices=list(previous_tickers), visible=True),
            gr.update(visible=False)
        )

    # def reset_fields():
    #     return "", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)

    def reset_fields():
        return "", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)


    # def update_ticker_input(selected_ticker):
    #     return selected_ticker, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(value="", visible=False)

    # def auto_submit_ticker(selected_ticker, previous_tickers):
    #     return update_visibility(selected_ticker, previous_tickers)
        
    def update_ticker_input(selected_ticker):
        return selected_ticker, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(value="", visible=False), gr.update(visible=False)

    def auto_submit_ticker(selected_ticker, previous_tickers):
        return update_visibility(selected_ticker, previous_tickers)

    submit_btn.click(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])
    ticker_input.submit(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])
    # reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message])
    reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message, chart_output2])
    # previous_tickers_display.change(update_ticker_input, inputs=previous_tickers_display, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message])
    # previous_tickers_display.change(auto_submit_ticker, inputs=[previous_tickers_display, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])

    previous_tickers_display.change(update_ticker_input, inputs=previous_tickers_display, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message, chart_output2])
    previous_tickers_display.change(auto_submit_ticker, inputs=[previous_tickers_display, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])

# Gradio 앱 실행
demo.launch()
# demo.launch(share=True)

HTTP 에러 발생: 405 Client Error: METHOD NOT ALLOWED for url: http://121.162.37.155:5000/predict


NameError: name 'modal_button' is not defined

## 모바일 화면 최적화 적용

In [ ]:
# 백엔드 데이터 연동
from backend.stockchart import *
from backend.exchangerate import *
from gradio_modal import Modal

daily = dailyChart()
daily[0].head(10)
daily[1].head(10)

# 사용 가능한 티커 목록 가져오기
available_tickers = pd.read_csv('../data/available_tickers.csv')['tickers'].str.upper().tolist()
random.shuffle(available_tickers)

# 이미지 파일을 base64로 인코딩
with open("images/bg_01.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode()

with open("images/up_kr.jpeg", "rb") as image_file:
    encoded_up_image = base64.b64encode(image_file.read()).decode()

with open("images/down_kr.jpeg", "rb") as image_file:
    encoded_down_image = base64.b64encode(image_file.read()).decode()
    
# Gradio 인터페이스 설정
css_code = f"""
.gradio-container {{
    background-image: url(data:image/png;base64,{encoded_string});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 0.8;
}}

.custom-markdown h1, p {{
    color: white;
    text-align: center;
}}

div.custom-title > div > p {{
    color: white;
}}

div.small-font {{
    height: 170px !important;
    overflow-y: auto !important;
}}

div.small-font span {{
    font-size: 12px;
}}

.table-wrap {{
    max-height: 400px !important;
    overflow-y: auto !important;
}}

.predict_wrapper {{
    display: flex;
    justify-content: space-between;
}}

.predict_result {{
    width: 50%;
    height: 200px;
    background-color: white;
    border-radius: 10px;
    margin-left: 10px;
    padding-bottom: 5px;
}}

.predict_result_up {{
    width: 50%;
    height: 200px;
    border-radius: 10px;
    background-image: url(data:image/png;base64,{encoded_up_image});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 1;
}}

.predict_result_down {{
    width: 50%;
    height: 200px;
    border-radius: 10px;
    background-image: url(data:image/png;base64,{encoded_down_image});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 1;
}}

.predict_ticker, .predict_price, .start_price {{
    padding: 10px;
    font-size: 20px;
}}

span.price_down {{
    font-size: 3rem;
    color: blue;
    font-weight: 700;
}}

span.price_up {{
    font-size: 3rem;
    color: red;
    font-weight: 700;
}}
"""

import requests

# 요청 URL
url = "http://121.162.37.155:5000/predict"

# 요청 헤더 설정
headers = {
    "Content-Type": "application/json"
}

# 요청 본문 설정 (필요한 데이터로 수정)
payload = {
    # API에서 요구하는 데이터를 여기에 추가하세요.
    "ticker": "AAPL"
}

try:
    # POST 요청 보내기
    response = requests.get(url, json=payload, headers=headers)
    
    # 요청이 성공했는지 확인
    response.raise_for_status()
    
    # JSON 응답 받기
    json_response = response.json()
    print("응답 받은 JSON:", json_response)
    
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP 에러 발생: {http_err}")
except Exception as err:
    print(f"기타 에러 발생: {err}")

import plotly.graph_objects as go

# 주식 데이터를 가져와 Plotly 차트로 변환하는 함수
def get_stock_chart(ticker):
    # 주식 데이터 다운로드 (최근 6개월)
    data, img = get_yfinance_stock_data(ticker)

    data = data.sort_values(by="Date")
    
    # Plotly 라인 차트 생성
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data["Date"], y=data["Close"], mode="lines", name="종가"))
    fig.add_trace
    fig.update_layout(
        title=f"{ticker.upper()} 주식 종가 라인 차트",
        xaxis_title="날짜",
        yaxis_title="가격",
        template="plotly_white"  # 밝은 테마
    )

    return fig

def info_fn():
    information="""
    [오른쪽 X를 눌러 창 닫기]<br>
    < 분석 기준 ><br>
    - open(시작가)<br>
    - close(종가)<br>
    - InterestRate(금리)<br>
    - ExchangeRate(1유로 기준 달러 환율)<br>
    - VIX(변동성 지수(=공포 지수))<br>
    - TEDSpread(은행 간 대출 금리와 단기 미국 국채 금리 간의 차이)<br>
    - EFFR(예금 기관들이 서로에게 하룻밤 동안 예비 잔고를 대출할때 적용되는 금리)<br>
    - Gold(금 가격)<br>
    - Oil(유가)<br>
    - Close_InterestRate_Corr(종가와 이자율의 상관계수)<br>
    - Close_VIX_Corr(종가-공포지수 상관계수)<br>
    - Daily_Return(일일 수익률)<br>
    - Volatility(30일 이동평균편차)<br>
    - Rolling_Mean_Close(30일 이동평균)<br><br>

    < 분석 기준 출처 ><br>
    - open,close,Gold, Oil = YFinance<br>
    - InterestRate, ExchangeRate, VIX,<br> TEDSpread, EFFR = FRED<br>
    - 달러 환율=네이버 은행고시환율(하나은행)<br><br>

    < 분석 기간 ><br>
    2010-01-01 ~ 2024-11-06<br><br>

    < 분석 모델 ><br> 
    RandomForestRegression<br><br>
    """
    gr.Info(information,duration=None,title="")

def close_modal():
    return gr.update(visible=False)

with gr.Blocks(gr.themes.Monochrome(), css=css_code) as demo:

    with Modal(visible=True,allow_user_close=False) as modal:
            gr.Image("images/start.png")
            agree_btn = gr.Button("START")

    agree_btn.click(close_modal, None, modal)

    exchange_rate = exchangerate()

    with gr.Row():
        trigger_info = gr.Button(value="분석 관련 정보")
        gr.Markdown("# 천하제일 단타대회", elem_classes="custom-markdown")
        gr.Markdown(f"1 USD: {exchange_rate}", elem_classes="custom-markdown")
        
    trigger_info.click(info_fn, None, None)
    modal_button.click(lambda: modal.show(), None, None)

    daily = dailyChart()
    volumn_top10 = daily[0].head(10)
    percent_top10 = daily[1].head(10)

    with gr.Row():
        with gr.Column(scale=1):
            gr.DataFrame(percent_top10, label="실시간 상승 주식", elem_classes="custom-title small-font")
        with gr.Column(scale=1):
            gr.DataFrame(volumn_top10, label="실시간 거래량 많은 주식", elem_classes="custom-title small-font")

    previous_tickers = gr.State(set())
    gr.Markdown("# 주식 가격 예측", elem_classes="custom-markdown")
    previous_tickers_display = gr.Radio(label="이전 조회된 티커", choices=[], interactive=True, visible=False)
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder=f"예: {", ".join(available_tickers[:5])} 등")
    warning_message = gr.Markdown(value="", visible=False)
    with gr.Row():
        submit_btn = gr.Button("조회")
        reset_btn = gr.Button("초기화")

    with gr.Row():
        predict_result_up = gr.HTML(
            """
            <div class='predict_wrapper'>
                <div class='predict_result_up'>

                </div>
                <div class='predict_result'>
                    {ticker}
                </div>
            </div>
            """,
            visible=False
        )
        predict_result_up = gr.HTML(
            """
            <div class='predict_wrapper'>
                <div class='predict_result_up'>

                </div>
                <div class='predict_result'>
                    {ticker}
                </div>
            </div>
            """,
            visible=False
        )

        predict_result_up = gr.HTML(
            """
            <div class='predict_wrapper'>
                <div class='predict_result_up'>

                </div>
                <div class='predict_result'>
                    {ticker}
                </div>
            </div>
            """,
            visible=False
        )

        predict_result_down = gr.HTML(
            """
            <div class='predict_wrapper'>
                <div class='predict_result_down'>

                </div>
                <div class='predict_result'>
                    {ticker}
                </div>
            </div>
            """,
            visible=False
        )

    chart_output2 = gr.Plot(label="주식 차트", visible=False)
    
    # Gradio에서 입력과 출력을 연결
    ticker_input.submit(get_stock_chart, inputs=ticker_input, outputs=chart_output2)
    previous_tickers_display.change(update_ticker_input, inputs=previous_tickers_display, outputs=chart_output2)

    chart_output = gr.Image(label="주가 차트", elem_classes="custom-markdown", visible=False)
    data_output = gr.Dataframe(label="일별 주가 데이터", elem_classes="custom-title", visible=False)

    def update_visibility(ticker, previous_tickers):
        if not ticker:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="티커를 입력하세요.", visible=True, elem_classes="custom-markdown")

        ticker = ticker.upper()  # Convert ticker to uppercase
        if ticker.upper() not in available_tickers:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="입력하신 종목의 가격 예측은 추후에 제공될 예정입니다..", visible=True)

        data, img, isUp, open, close, prediction = predict_stock_price3(ticker)
        open = round(open, 2)
        close = round(close, 2)

        previous_tickers.add(ticker)
        return (
            gr.update(visible=True, label=f"일별 주가 데이터 [{ticker}]", elem_classes="custom-title"),
            gr.update(visible=False),
            gr.update(visible=isUp, value=f"""
            <div class='predict_wrapper'>
                <div class='predict_result_up'>

                </div>
                <div class='predict_result'>
                    <div class='predict_ticker'>
                        [ {ticker} ] 시가 : {open}
                    </div>  
                    <div class='predict_price'>
                        예측 종가: {prediction}<br><span class='price_up'>{close}(▲)</span>
                    </div>
                </div>
            </div>
            """),
            gr.update(visible=not isUp, value=f"""
            <div class='predict_wrapper'>
                <div class='predict_result_down'>

                </div>
                <div class='predict_result'>
                    <div class='predict_ticker'>
                        [ {ticker} ] 시가 : {open}
                    </div>  
                    <div class='predict_price'>
                        예측 종가: {prediction}<br><span class='price_down'>{close}(▼)</span>
                    </div>
                </div>
            </div>
            """),
            data,
            gr.update(visible=True),
            gr.update(choices=list(previous_tickers), visible=True),
            gr.update(visible=False)
        )

    def reset_fields():
        return "", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)

    def update_ticker_input(selected_ticker):
        return selected_ticker, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(value="", visible=False), gr.update(visible=False)

    def auto_submit_ticker(selected_ticker, previous_tickers):
        return update_visibility(selected_ticker, previous_tickers)

    submit_btn.click(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])
    ticker_input.submit(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])
    reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message, chart_output2])
    previous_tickers_display.change(update_ticker_input, inputs=previous_tickers_display, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message, chart_output2])
    previous_tickers_display.change(auto_submit_ticker, inputs=[previous_tickers_display, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])

# Gradio 앱 실행
demo.launch()
# demo.launch(share=True)

In [ ]:
# 백엔드 데이터 연동
from backend.stockchart import *
from backend.exchangerate import *
from gradio_modal import Modal

daily = dailyChart()
daily[0].head(10)
daily[1].head(10)

# 사용 가능한 티커 목록 가져오기
available_tickers = pd.read_csv('../data/available_tickers.csv')['tickers'].str.upper().tolist()
random.shuffle(available_tickers)

# 이미지 파일을 base64로 인코딩
with open("images/bg_01.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode()

with open("images/up_kr.jpeg", "rb") as image_file:
    encoded_up_image = base64.b64encode(image_file.read()).decode()

with open("images/down_kr.jpeg", "rb") as image_file:
    encoded_down_image = base64.b64encode(image_file.read()).decode()
    
# Gradio 인터페이스 설정
css_code = f"""
.gradio-container {{
    background-image: url(data:image/png;base64,{encoded_string});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 0.8;
}}

.custom-markdown h1, p {{
    color: white;
    text-align: center;
}}

div.custom-title > div > p {{
    color: white;
}}

div.small-font {{
    height: 170px !important;
    overflow-y: auto !important;
}}

div.small-font span {{
    font-size: 12px;
}}

.table-wrap {{
    max-height: 400px !important;
    overflow-y: auto !important;
}}

.predict_wrapper {{
    display: flex;
    justify-content: space-between;
}}

.predict_result {{
    width: 50%;
    height: 200px;
    background-color: white;
    border-radius: 10px;
    margin-left: 10px;
    padding-bottom: 5px;
}}

.predict_result_up {{
    width: 50%;
    height: 200px;
    border-radius: 10px;
    background-image: url(data:image/png;base64,{encoded_up_image});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 1;
}}

.predict_result_down {{
    width: 50%;
    height: 200px;
    border-radius: 10px;
    background-image: url(data:image/png;base64,{encoded_down_image});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 1;
}}

.predict_ticker, .predict_price, .start_price {{
    padding: 10px;
    font-size: 20px;
}}

span.price_down {{
    font-size: 3rem;
    color: blue;
    font-weight: 700;
}}

span.price_up {{
    font-size: 3rem;
    color: red;
    font-weight: 700;
}}
"""

import requests

# 요청 URL
url = "http://121.162.37.155:5000/predict"

# 요청 헤더 설정
headers = {
    "Content-Type": "application/json"
}

# 요청 본문 설정 (필요한 데이터로 수정)
payload = {
    # API에서 요구하는 데이터를 여기에 추가하세요.
    "ticker": "AAPL"
}

try:
    # POST 요청 보내기
    response = requests.get(url, json=payload, headers=headers)
    
    # 요청이 성공했는지 확인
    response.raise_for_status()
    
    # JSON 응답 받기
    json_response = response.json()
    print("응답 받은 JSON:", json_response)
    
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP 에러 발생: {http_err}")
except Exception as err:
    print(f"기타 에러 발생: {err}")

import plotly.graph_objects as go

# 주식 데이터를 가져와 Plotly 차트로 변환하는 함수
def get_stock_chart(ticker):
    # 주식 데이터 다운로드 (최근 6개월)
    data, img = get_yfinance_stock_data(ticker)

    data = data.sort_values(by="Date")
    
    # Plotly 라인 차트 생성
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data["Date"], y=data["Close"], mode="lines", name="종가"))
    fig.add_trace
    fig.update_layout(
        title=f"{ticker.upper()} 주식 종가 라인 차트",
        xaxis_title="날짜",
        yaxis_title="가격",
        template="plotly_white"  # 밝은 테마
    )

    return fig

def info_fn():
    information="""
    [오른쪽 X를 눌러 창 닫기]<br>
    < 분석 기준 ><br>
    - open(시작가)<br>
    - close(종가)<br>
    - InterestRate(금리)<br>
    - ExchangeRate(1유로 기준 달러 환율)<br>
    - VIX(변동성 지수(=공포 지수))<br>
    - TEDSpread(은행 간 대출 금리와 단기 미국 국채 금리 간의 차이)<br>
    - EFFR(예금 기관들이 서로에게 하룻밤 동안 예비 잔고를 대출할때 적용되는 금리)<br>
    - Gold(금 가격)<br>
    - Oil(유가)<br>
    - Close_InterestRate_Corr(종가와 이자율의 상관계수)<br>
    - Close_VIX_Corr(종가-공포지수 상관계수)<br>
    - Daily_Return(일일 수익률)<br>
    - Volatility(30일 이동평균편차)<br>
    - Rolling_Mean_Close(30일 이동평균)<br><br>

    < 분석 기준 출처 ><br>
    - open,close,Gold, Oil = YFinance<br>
    - InterestRate, ExchangeRate, VIX,<br> TEDSpread, EFFR = FRED<br>
    - 달러 환율=네이버 은행고시환율(하나은행)<br><br>

    < 분석 기간 ><br>
    2010-01-01 ~ 2024-11-06<br><br>

    < 분석 모델 ><br> 
    RandomForestRegression<br><br>
    """
    gr.Info(information,duration=None,title="")

def close_modal():
    return gr.update(visible=False)

with gr.Blocks(gr.themes.Monochrome(), css=css_code) as demo:

    with Modal(visible=True,allow_user_close=False) as modal:
            gr.Image("images/start.png")
            agree_btn = gr.Button("START")

    agree_btn.click(close_modal, None, modal)

    exchange_rate = exchangerate()

    with gr.Row():
        with gr.Column(scale=1):
            trigger_info = gr.Button(value="분석 관련 정보")
        with gr.Column(scale=1):
            gr.Markdown("# 천하제일 단타대회", elem_classes="custom-markdown")
        with gr.Column(scale=1):
            gr.Markdown(f"1 USD: {exchange_rate}", elem_classes="custom-markdown")
        
    trigger_info.click(info_fn, None, None)

    daily = dailyChart()
    volumn_top10 = daily[0].head(10)
    percent_top10 = daily[1].head(10)

    with gr.Row():
        with gr.Column(scale=1):
            gr.DataFrame(percent_top10, label="실시간 상승 주식", elem_classes="custom-title small-font")
        with gr.Column(scale=1):
            gr.DataFrame(volumn_top10, label="실시간 거래량 많은 주식", elem_classes="custom-title small-font")

    previous_tickers = gr.State(set())
    gr.Markdown("# 주식 가격 예측", elem_classes="custom-markdown")
    previous_tickers_display = gr.Radio(label="이전 조회된 티커", choices=[], interactive=True, visible=False)
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder=f"예: {", ".join(available_tickers[:5])} 등")
    warning_message = gr.Markdown(value="", visible=False)
    with gr.Row():
        submit_btn = gr.Button("조회")
        reset_btn = gr.Button("초기화")

    with gr.Row():
        predict_result_up = gr.HTML(
            """
            <div class='predict_wrapper'>
                <div class='predict_result_up'>

                </div>
                <div class='predict_result'>
                    {ticker}
                </div>
            </div>
            """,
            visible=False
        )
        predict_result_up = gr.HTML(
            """
            <div class='predict_wrapper'>
                <div class='predict_result_up'>

                </div>
                <div class='predict_result'>
                    {ticker}
                </div>
            </div>
            """,
            visible=False
        )

        predict_result_up = gr.HTML(
            """
            <div class='predict_wrapper'>
                <div class='predict_result_up'>

                </div>
                <div class='predict_result'>
                    {ticker}
                </div>
            </div>
            """,
            visible=False
        )

        predict_result_down = gr.HTML(
            """
            <div class='predict_wrapper'>
                <div class='predict_result_down'>

                </div>
                <div class='predict_result'>
                    {ticker}
                </div>
            </div>
            """,
            visible=False
        )

    chart_output2 = gr.Plot(label="주식 차트", visible=False)
    
    # Gradio에서 입력과 출력을 연결
    ticker_input.submit(get_stock_chart, inputs=ticker_input, outputs=chart_output2)
    previous_tickers_display.change(update_ticker_input, inputs=previous_tickers_display, outputs=chart_output2)

    chart_output = gr.Image(label="주가 차트", elem_classes="custom-markdown", visible=False)
    data_output = gr.Dataframe(label="일별 주가 데이터", elem_classes="custom-title", visible=False)

    def update_visibility(ticker, previous_tickers):
        if not ticker:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="티커를 입력하세요.", visible=True, elem_classes="custom-markdown")

        ticker = ticker.upper()  # Convert ticker to uppercase
        # if ticker.upper() not in available_tickers:
        #     return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="입력하신 종목의 가격 예측은 추후에 제공될 예정입니다..", visible=True)

        data, img, isUp, open, close, prediction, max_diff, min_diff = predict_stock_price3(ticker)
        open = round(open, 2)
        close = round(close, 2)
        max_diff = round(max_diff, 2)
        min_diff = round(min_diff, 2)

        previous_tickers.add(ticker)
        return (
            gr.update(visible=True, label=f"일별 주가 데이터 [{ticker}]", elem_classes="custom-title"),
            gr.update(visible=False),
            gr.update(visible=isUp, value=f"""
            <div class='predict_wrapper'>
                <div class='predict_result_up'>

                </div>
                <div class='predict_result'>
                    <div class='predict_ticker'>
                        [ {ticker} ] 시가 : {open}
                    </div>  
                    <div class='predict_price'>
                        예측 종가: {prediction}<br><span class='price_up'>{close}(▲)</span><br>
                        10거래일 수익률 max[{max_diff}%] min[{min_diff}%]<br>
                    </div>
                </div>
            </div>
            """),
            gr.update(visible=not isUp, value=f"""
            <div class='predict_wrapper'>
                <div class='predict_result_down'>

                </div>
                <div class='predict_result'>
                    <div class='predict_ticker'>
                        [ {ticker} ] 시가 : {open}
                    </div>  
                    <div class='predict_price'>
                        예측 종가: {prediction}<br><span class='price_down'>{close}(▼)</span><br>
                        10거래일 수익률 max[{max_diff}%] min[{min_diff}%]<br>
                    </div>
                </div>
            </div>
            """),
            data,
            gr.update(visible=True),
            gr.update(choices=list(previous_tickers), visible=True),
            gr.update(visible=False)
        )

    def reset_fields():
        return "", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)

    def update_ticker_input(selected_ticker):
        fig = get_stock_chart(selected_ticker)
        return selected_ticker, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(value="", visible=False), gr.update(visible=False), fig

    # def auto_submit_ticker(selected_ticker, previous_tickers):
    #     return update_visibility(selected_ticker, previous_tickers)      

    def auto_submit_ticker(selected_ticker, previous_tickers):
        return update_visibility(selected_ticker, previous_tickers)

    submit_btn.click(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])
    ticker_input.submit(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])
    reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message, chart_output2])
    # previous_tickers_display.change(update_ticker_input, inputs=previous_tickers_display, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message, chart_output2])
    # previous_tickers_display.change(auto_submit_ticker, inputs=[previous_tickers_display, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])

    # 이벤트 바인딩 수정
    previous_tickers_display.change(
        update_ticker_input, 
        inputs=previous_tickers_display, 
        outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message, previous_tickers_display, chart_output2]
    )

    previous_tickers_display.change(
        auto_submit_ticker, 
        inputs=[previous_tickers_display, previous_tickers], 
        outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message]
    )

# Gradio 앱 실행
demo.launch()
# demo.launch(share=True)

HTTP 에러 발생: 405 Client Error: METHOD NOT ALLOWED for url: http://121.162.37.155:5000/predict
* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


[*********************100%***********************]  1 of 1 completed

/Users/cool/msai_t6/common/yfinance.py:50: UserWarning:

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.



응답 받은 JSON: {'Max_diff': 0.4012246362753026, 'Min_diff': 0.007355078793687666, 'Open': 224.625, 'Predicted Close': 224.65625190734863, 'Prediction': 'UP'}


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


응답 받은 JSON: {'Max_diff': 1.1726920268304193, 'Min_diff': 0.07657747010926944, 'Open': 898.5, 'Predicted Close': 903.8396911621094, 'Prediction': 'UP'}


In [ ]:
# 백엔드 데이터 연동
from backend.stockchart import *
from backend.exchangerate import *
from gradio_modal import Modal

daily = dailyChart()
daily[0].head(10)
daily[1].head(10)

# 사용 가능한 티커 목록 가져오기
available_tickers = pd.read_csv('../data/available_tickers.csv')['tickers'].str.upper().tolist()
random.shuffle(available_tickers)

# 이미지 파일을 base64로 인코딩
with open("images/bg_01.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode()

with open("images/up.png", "rb") as image_file:
    encoded_up_image = base64.b64encode(image_file.read()).decode()

with open("images/down.png", "rb") as image_file:
    encoded_down_image = base64.b64encode(image_file.read()).decode()
    
# Gradio 인터페이스 설정
css_code = f"""
.gradio-container {{
    background-image: url(data:image/png;base64,{encoded_string});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 0.8;
}}

.custom-markdown h1, p {{
    color: white;
    text-align: center;
}}

div.custom-title > div > p {{
    color: white;
}}

div.small-font {{
    height: 170px !important;
    overflow-y: auto !important;
}}

div.small-font::-webkit-scrollbar {{
  display: none;
}}

div.small-font span {{
    font-size: 12px;
}}

.table-wrap {{
    max-height: 400px !important;
    overflow-y: auto !important;
}}

.table-wrap::-webkit-scrollbar {{
  display: none;
}}

.predict_wrapper {{
    display: flex;
    justify-content: space-between;
}}

.predict_result {{
    width: 50%;
    height: 300px;
    background-color: white;
    border-radius: 10px;
    margin-left: 10px;
    padding-bottom: 5px;
}}

.predict_result_up {{
    width: 50%;
    height: 300px;
    border-radius: 10px;
    background-image: url(data:image/png;base64,{encoded_up_image});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 1;
}}

.predict_result_down {{
    width: 50%;
    height: 300px;
    border-radius: 10px;
    background-image: url(data:image/png;base64,{encoded_down_image});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 1;
}}

.predict_ticker, .predict_price, .start_price {{
    padding: 20px 10px 20px 10px;
    font-size: 20px;
}}

span.price_down {{
    font-size: 3rem;
    color: blue;
    font-weight: 700;
}}

span.price_up {{
    font-size: 3rem;
    color: red;
    font-weight: 700;
}}

.max_diff {{
    font-size: 1.5rem;
    font-weight: 700;
    color: yellow;
    padding: 2px;
    rounded: 2px;
    background-color: red;
}}

.min_diff {{
    font-size: 1.5rem;
    font-weight: 700;
    color: white;
    padding: 2px;
    rounded: 2px;
    background-color: blue;
}}
"""

import requests

# 요청 URL
url = "http://121.162.37.155:5000/predict"

# 요청 헤더 설정
headers = {
    "Content-Type": "application/json"
}

# 요청 본문 설정 (필요한 데이터로 수정)
payload = {
    # API에서 요구하는 데이터를 여기에 추가하세요.
    "ticker": "AAPL"
}

try:
    # POST 요청 보내기
    response = requests.get(url, json=payload, headers=headers)
    
    # 요청이 성공했는지 확인
    response.raise_for_status()
    
    # JSON 응답 받기
    json_response = response.json()
    print("응답 받은 JSON:", json_response)
    
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP 에러 발생: {http_err}")
except Exception as err:
    print(f"기타 에러 발생: {err}")

import plotly.graph_objects as go

# 주식 데이터를 가져와 Plotly 차트로 변환하는 함수
def get_stock_chart(ticker):
    # 주식 데이터 다운로드 (최근 6개월)
    data, img = get_yfinance_stock_data(ticker)

    data = data.sort_values(by="Date")
    
    # Plotly 라인 차트 생성
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data["Date"], y=data["Close"], mode="lines", name="종가"))
    fig.add_trace
    fig.update_layout(
        title=f"{ticker.upper()} 주식 종가 라인 차트",
        xaxis_title="Date",
        yaxis_title="Close Price",
        template="plotly_white"  # 밝은 테마
    )

    return fig

def info_fn():
    information="""
    [오른쪽 X를 눌러 창 닫기]<br>
    < 분석 기준 ><br>
    - open(시작가)<br>
    - close(종가)<br>
    - InterestRate(금리)<br>
    - ExchangeRate(1유로 기준 달러 환율)<br>
    - VIX(변동성 지수(=공포 지수))<br>
    - TEDSpread(은행 간 대출 금리와 단기 미국 국채 금리 간의 차이)<br>
    - EFFR(예금 기관들이 서로에게 하룻밤 동안 예비 잔고를 대출할때 적용되는 금리)<br>
    - Gold(금 가격)<br>
    - Oil(유가)<br>
    - Close_InterestRate_Corr(종가와 이자율의 상관계수)<br>
    - Close_VIX_Corr(종가-공포지수 상관계수)<br>
    - Daily_Return(일일 수익률)<br>
    - Volatility(30일 이동평균편차)<br>
    - Rolling_Mean_Close(30일 이동평균)<br><br>

    < 분석 기준 출처 ><br>
    - open,close,Gold, Oil = YFinance<br>
    - InterestRate, ExchangeRate, VIX,<br> TEDSpread, EFFR = FRED<br>
    - 달러 환율=네이버 은행고시환율(하나은행)<br><br>

    < 분석 기간 ><br>
    2010-01-01 ~ 2024-11-07<br><br>

    < 분석 모델 ><br> 
    LGBMRegressor<br><br>
    """
    gr.Info(information,duration=None,title="")

def close_modal():
    return gr.update(visible=False)

with gr.Blocks(gr.themes.Monochrome(), css=css_code) as demo:

    with Modal(visible=True,allow_user_close=False) as modal:
            gr.Image("images/start_mobile.png", show_label=False, show_download_button=False, show_fullscreen_button=False)
            agree_btn = gr.Button("START")

    agree_btn.click(close_modal, None, modal)

    exchange_rate = exchangerate()

    with gr.Row():
        trigger_info = gr.Button(value="분석 관련 정보")
        gr.Markdown("# 천하제일 단타대회", elem_classes="custom-markdown")
        gr.Markdown(f"1 USD: {exchange_rate}", elem_classes="custom-markdown")
        
    trigger_info.click(info_fn, None, None)

    daily = dailyChart()
    volumn_top10 = daily[0].head(10)
    percent_top10 = daily[1].head(10)

    with gr.Row():
        with gr.Column(scale=1):
            gr.DataFrame(percent_top10, label="실시간 상승 주식", elem_classes="custom-title small-font")
        with gr.Column(scale=1):
            gr.DataFrame(volumn_top10, label="실시간 거래량 많은 주식", elem_classes="custom-title small-font")

    previous_tickers = gr.State(set())
    gr.Markdown("# 주식 가격 예측", elem_classes="custom-markdown")
    previous_tickers_display = gr.Radio(label="이전 조회된 티커", choices=[], interactive=True, visible=False)
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder=f"예: {", ".join(available_tickers[:5])} 등")
    warning_message = gr.Markdown(value="", visible=False)
    with gr.Row():
        submit_btn = gr.Button("조회")
        reset_btn = gr.Button("초기화")

    with gr.Row():
        predict_result_up = gr.HTML(
            """
            <div class='predict_wrapper'>
                <div class='predict_result_up'>

                </div>
                <div class='predict_result'>
                    {ticker}
                </div>
            </div>
            """,
            visible=False
        )
        predict_result_up = gr.HTML(
            """
            <div class='predict_wrapper'>
                <div class='predict_result_up'>

                </div>
                <div class='predict_result'>
                    {ticker}
                </div>
            </div>
            """,
            visible=False
        )

        predict_result_up = gr.HTML(
            """
            <div class='predict_wrapper'>
                <div class='predict_result_up'>

                </div>
                <div class='predict_result'>
                    {ticker}
                </div>
            </div>
            """,
            visible=False
        )

        predict_result_down = gr.HTML(
            """
            <div class='predict_wrapper'>
                <div class='predict_result_down'>

                </div>
                <div class='predict_result'>
                    {ticker}
                </div>
            </div>
            """,
            visible=False
        )

    chart_output2 = gr.Plot(label="주식 차트", visible=False)
    
    chart_output = gr.Image(label="주가 차트", elem_classes="custom-markdown", visible=False)
    data_output = gr.Dataframe(label="일별 주가 데이터", elem_classes="custom-title", visible=False)

    def update_visibility(ticker, previous_tickers):
        if not ticker:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="티커를 입력하세요.", visible=True, elem_classes="custom-markdown")

        ticker = ticker.upper()  # Convert ticker to uppercase
        # if ticker.upper() not in available_tickers:
        #     return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="입력하신 종목의 가격 예측은 추후에 제공될 예정입니다..", visible=True)

        data, img, isUp, open, close, prediction, max_diff, min_diff = predict_stock_price3(ticker)
        open = round(open, 2)
        close = round(close, 2)
        max_diff = round(max_diff, 2)
        min_diff = round(min_diff, 2)

        previous_tickers.add(ticker)
        return (
            gr.update(visible=True, label=f"일별 주가 데이터 [{ticker}]", elem_classes="custom-title"),
            gr.update(visible=False),
            gr.update(visible=isUp, value=f"""
            <div class='predict_wrapper'>
                <div class='predict_result_up'>

                </div>
                <div class='predict_result'>
                    <div class='predict_ticker'>
                        [ {ticker} ] 시가 : {open}
                    </div>  
                    <div class='predict_price'>
                        예측 종가: {prediction}<br><span class='price_up'>{close}(▲)</span><br>
                        10거래일 수익률 <span class='max_diff'>{max_diff}%</span>~<span class='min_diff'>{min_diff}%</span>
                    </div>
                </div>
            </div>
            """),
            gr.update(visible=not isUp, value=f"""
            <div class='predict_wrapper'>
                <div class='predict_result_down'>

                </div>
                <div class='predict_result'>
                    <div class='predict_ticker'>
                        [ {ticker} ] 시가 : {open}
                    </div>  
                    <div class='predict_price'>
                        예측 종가: {prediction}<br><span class='price_down'>{close}(▼)</span><br>
                        10거래일 수익률 <span class='max_diff'>{max_diff}%</span>~<span class='min_diff'>{min_diff}%</span>
                    </div>
                </div>
            </div>
            """),
            data,
            gr.update(visible=True),
            gr.update(choices=list(previous_tickers), visible=True),
            gr.update(visible=False)
        )

    def reset_fields():
        return "", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)

    def update_ticker_input(selected_ticker):
        fig = get_stock_chart(selected_ticker)
        return selected_ticker, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(value="", visible=False), gr.update(visible=False), fig

    # def auto_submit_ticker(selected_ticker, previous_tickers):
    #     return update_visibility(selected_ticker, previous_tickers)      

    def auto_submit_ticker(selected_ticker, previous_tickers):
        return update_visibility(selected_ticker, previous_tickers)

    # Gradio에서 입력과 출력을 연결
    ticker_input.submit(get_stock_chart, inputs=ticker_input, outputs=chart_output2)
    previous_tickers_display.change(update_ticker_input, inputs=previous_tickers_display, outputs=chart_output2)

    submit_btn.click(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])
    ticker_input.submit(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])
    reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message, chart_output2])
    # previous_tickers_display.change(update_ticker_input, inputs=previous_tickers_display, outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message, chart_output2])
    # previous_tickers_display.change(auto_submit_ticker, inputs=[previous_tickers_display, previous_tickers], outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message])

    # 이벤트 바인딩 수정
    previous_tickers_display.change(
        update_ticker_input, 
        inputs=previous_tickers_display, 
        outputs=[ticker_input, data_output, chart_output, predict_result_up, predict_result_down, warning_message, previous_tickers_display, chart_output2]
    )

    previous_tickers_display.change(
        auto_submit_ticker, 
        inputs=[previous_tickers_display, previous_tickers], 
        outputs=[data_output, chart_output, predict_result_up, predict_result_down, data_output, chart_output2, previous_tickers_display, warning_message]
    )

# Gradio 앱 실행
demo.launch()
# demo.launch(share=True)

HTTP 에러 발생: 405 Client Error: METHOD NOT ALLOWED for url: http://121.162.37.155:5000/predict
* Running on local URL:  http://127.0.0.1:7891

To create a public link, set `share=True` in `launch()`.
